In [19]:
import re
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz, process

NOT_HIT_TEMPLATE_KEY = '__empty__'


class SMSLoanHistoryAnalysisConfig:
    _category_config = None
    _template_config = None
    _phone_regex = None

    app_phone_list = [
        {'loan_app': 'cashbean', 'phone_regex': re.compile('.*?(CASHBN|CASHME)$')},
        {'loan_app': 'kreditbee', 'phone_regex': re.compile('.*?(KREDTB|KRBEEE)$')},
        {'loan_app': 'cashmama', 'phone_regex': re.compile('.*?(CSHMMA|DIALST)$')},
        {'loan_app': 'rupeemax', 'phone_regex': re.compile('.*?(?i)RupMax$')},
        {'loan_app': 'nanocred', 'phone_regex': re.compile('.*?(?i)(NANOCD|NANOCR)$')},
        {'loan_app': 'kissht', 'phone_regex': re.compile('.*?(?i)KISSHT$')},
        {'loan_app': 'cashbus', 'phone_regex': re.compile('.*?(?i)(CASHBS|CBLOAN)$')},
        {'loan_app': 'gotocash', 'phone_regex': re.compile('.*?(?i)GTCash$')},
        {'loan_app': 'kreditzy', 'phone_regex': re.compile('.*?(?i)KREDTZ$')},
        {'loan_app': 'zestmoney', 'phone_regex': re.compile('.*?(?i)(ZESTMN|ZESTMO)$')},
        {'loan_app': 'moneed', 'phone_regex': re.compile('.*?(?i)(MONEED|Notice)$')}

    ]
    templates_list = [{'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear kreditbee customer, please ensure that you make your installment payments on time to avoid penalties, reduction on bureau scores and to be eligible for higher credit limits. you can pay using bank transfer or upi for easy payments. ignore this message if already paid.'},
                      {'loan_app': 'kreditbee',
                       'category': 'registration',
                       'sms_template': '<#> 684331 is otp for kreditbee signup. do not share otp with anyone vueeydujxhz'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear tom, payment of rs. 5224 is due by 2019-12-22. pay on time to avoid penalty of rs. 500. pay via bank transfer/debit card/netbanking/paytm wallet/upi from the options in kreditbee app. select bank transfer option to avoid convenience fees. ignore if paid. https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear rachita, payment of rs. 5185 is due by 2020-04-01. pay on time to avoid penalty of rs. 104. pay via bank transfer/debit card/netbanking/paytm wallet/upi from the options in kreditbee app. select bank transfer option to avoid convenience fees. ignore if paid. https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'repaid',
                       'sms_template': 'dear customer, we have successfully received payment of rs. 2602. please wait upto 24 hours for the amount to reflect in your loan'},
                      {'loan_app': 'kreditbee',
                       'category': 'repaid',
                       'sms_template': 'dear ankush, your loan kb191207tswqp has been closed successfully! kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'registration',
                       'sms_template': '<#> 352787 is otp for loan agreement digital signature. do not share code with anyone vueeydujxhz'},
                      {'loan_app': 'kreditbee',
                       'category': 'disbursal',
                       'sms_template': 'congratulations hemant! your loan has been disbursed to your bank account xxxxxxxxxxxx0916 after deduction of processing fees. repay on time to avoid late payment charges! kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'announcement',
                       'sms_template': 'dear kreditbee customer, due to corona virus lockdown, our call center is operating at reduced capacity. to get your queries resolved faster, please write to us on help@kreditbee.in'},
                      {'loan_app': 'kreditbee',
                       'category': 'apply',
                       'sms_template': 'dear hemant, we have received your loan application on kreditbee. please wait while we verify the details. note that kreditbee agents will never ask you to transfer money for verification or any other services.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear customer, kreditbee referral offer for you. get upto rs 150 off on processing fee in your next loan after your friend registering with your referral code. refer now. hurry! link: kredt.be/refer'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear more, payment of rs. 5185 is due tomorrow. pay on time to avoid penalty of rs. 104. pay via bank transfer/debit card/netbanking/paytm wallet/upi from the options in kreditbee app. select bank transfer option to avoid convenience fees. ignore if paid. https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'avail instant loan upto 2 lakh in 10 min on kreditbee. 100% online process. direct bank transfer. check eligibility on kreditbee now. hurry! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear ankit, your loan is overdue and charges of rs. 21 have been levied. amount repayable is rs. 1045. pay immediately to avoid further penalty. ignore if paid. https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'registration',
                       'sms_template': 'dear m saravanankumar, you have successfully signed up on kreditbee! keep your pan card ready to check your loan eligibility. kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'congratulations hemant, your credit has been upgraded! you can now avail loans upto rs. 8000 at kreditbee at lower interest rates. avail an instant loan now!'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'congratulations nilesh! you have discovered a lower pricing for your loans. avail an instant loan at kreditbee now! https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'congratulations! you are one step away to avail your next loan on kreditbee. 100% online process. avail loan now. link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'reject',
                       'sms_template': 'sorry! we could not approve your profile at this moment. please re-apply in 6 months'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear hemant, please ensure that your cibil score is not impacted by paying rs. 2602 today. this will also help you to get more credit on kreditbee platform based on your good payment behavior and avoid late payment charges of rs. 52. pay via kreditbee {{link}}. ignore if paid.'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear pasham prashanth, please ensure that your cibil score is not impacted by paying rs. 5074 today. this will also help you to get more credit on kreditbee platform based on your good payment behavior and avoid late payment charges of rs. 100. ignore if paid. https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear more vijay, payment of rs. 5003 is due tomorrow. pay on time to avoid penalty of rs. 100. pay via bank transfer/debit card/netbanking/paytm wallet/upi from the options in kreditbee app. select bank transfer option to avoid convenience fees. ignore if paid. https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear praveen, please ensure that your cibil score is not impacted by paying rs. 4270 today. this will also help you to get more credit on kreditbee platform based on your good payment behavior and avoid late payment charges of rs. 500. ignore if paid. https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear dharmendra, please repay your emi amount due on 2020-03-08 as per following details using imps transfer. the details are only for you and transfer will close your loan only. do not share details with anyone.\n  beneficiary name: finnovation tech\n  acc. num: 888888018219116\n  ifsc: yesb0cmsnoc\n  remarks: loan emi payment\n  amount: 1795'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'legal notice alert - dear dharmendra, this is to inform you that your loan amount of rs. 1795 is overdue and pending payment since 18 days. hence, we request you to make the payment on priority. in case it is not done, you may receive a legal notice from us and we may initiate legal proceedings on your contract as per policy. please ignore if already paid. https://kredt.be/app - team kreditbee'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear saravanan, your bank account xxxxxx3999 has been verified successfully. submit your loan application for instant loan now! kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'apply',
                       'sms_template': 'dear anil, we have received your personal loan for salaried loan application. please wait upto 72 hours while we verify your details.'},
                      {'loan_app': 'kreditbee',
                       'category': 'reject',
                       'sms_template': 'dear nater! we could not verify your submitted information. please retry with the same or another option. https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear dharmendra, you can repay your outstanding emi amount of rs. 1467 due on 08-03-2020 using the given repayment link. pay via debit card, netbanking, wallets. convenience fees charges as applicable may apply. click the link to pay. https://krdit.be/200206kqrjh8c7nn'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear saravanan, your loan contract has a delinquency of rs. 10572 since 4 days. we request you to make the payment of the pending amount at the earliest. please ignore if already paid. https://kredt.be/app - team kreditbee'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': "refer and save money on your next kreditbee loan. refer your fruend on kreditbee and both of you get upto 100 rs. on processing fee. get rs. 50 off on your friend's kreditbee profile verification and rs. 50 on him taking a loan. refer more, save more. hurry! link: kredt.be/home"},
                      {'loan_app': 'kreditbee',
                       'category': 'apply',
                       'sms_template': 'dear tushar, we are verifying your profile. please wait upto 24 hours for the process to be completed. we will notify you once your profile is verified.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'get your next kreditbee loan in 2 mins. 100% online process. interest starting at 1.5% only. direct bank transfer. get loan now from kreditbee at http://nmc.sg/k9d48i'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'paytm cashback worth 20% of processing fees on your first kreditbee loan. offer valid till 30 nov midnight. hurry! get loan now. link: http://nmc.sg/kpmnjg'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'offer: get rs. 100 paytm cashback on your first kreditbee loan. complete your ekyc & avail loan now. valid till 7th feb. hurry! t&c apply* link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'apply',
                       'sms_template': 'dear ashwinkumar, we have received your loan application! please wait while we verify the details.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'loan upto rs. 2 lakh in kreditbee in 2 mins. interest starting at 1.2% only. 100% online process. direct bank transfer. get loan now. hurry! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'loan upto rs.1 lakh in 5 min. 100% online process. direct bank transfer. interest starting at 1.5% only. get loan now from kreditbee at http://nmc.sg/kfdb62'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'get upto rs. 100 off on processing fee of your next kreditbee loan on each referral. refer your friends to avail a kreditbee loan after registering with your referral code & both of you win 100 points. hurry! link: kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear customer, get flat rs. 150 paytm cashback on your next loan on kreditbee. avail loan now. offer valid till 30 sep only. hurry! link: https://kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear vivek, please ensure that your cibil score is not impacted by paying rs. 1816 today. this will also help you to get more credit on kreditbee platform based on your good payment behavior and avoid late payment charges of rs. 36. pay via kreditbee {{link}}. ignore if paid.'},
                      {'loan_app': 'kreditbee',
                       'category': 'NACH',
                       'sms_template': 'you have initiated a nach mandate registration with kreditbee (finnovation tech sol pvt. ltd.) for your account number ending xxxxxx6400 for loan repayments with monthly frequency. if you have not initiated the process of mandate registration, please call and report it to kreditbee customer care 08044292200.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'earn rs.1000 for referring 5 micro loans (<10k) on kreditbee. get rs.100 for each referral and rs.500 extra for every 5th referral. valid till 30 april. code - sarsxdm0x link - http://nmc.sg/dbguu8'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear customer, get loan upto 2 lakhs only in kreditbee at lower interest rate starting at 1.02% pm. complete your profile now! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear amit tak! flat rs.151 paytm cashback on your salary profile approval on kreditbee. get upto 120% of your salary with our premium salaried loans. complete salary profile now. valid till november 15. hurry! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear javed, please repay your emi amount due on 2019-11-23 as per following details using imps transfer. the details are only for you and transfer will close your loan only. do not share details with anyone.\n  beneficiary name: finnovation tech\n  acc. num: 888888007933879\n  ifsc: yesb0cmsnoc\n  remarks: 9726467867\n  amount: 3903'},
                      {'loan_app': 'kreditbee',
                       'category': 'registration',
                       'sms_template': 'dear lachutha, your uploaded aadhaar document(s) were not clear. please try again with a good quality original document. kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'loan upto 1 lakh in 15 mins. direct bank transfer. 100% online process. interest starting at 1.5% only. check eligibility on kreditbee now: https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'announcement',
                       'sms_template': 'dear kreditbee customer, bank transfer as a payment method will not be available temporarily due to industry-wide outage. please use debit card, netbanking, wallets or upi to make the repayment of your emi. remember to pay on time to avoid penalties and impact on cibil score.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'flat rs. 150 cashback on your salary profile verification on kreditbee. get higher loans with personal loan for salaried. valid till 10 dec! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': "refer friends on kreditbee app & win. get your friends registered on kreditbee app with referral code on the app & help them avail a loan. win rs.100 off on your next kreditbee loan. what's more? your friend wins 60 points on availing their first kreditbee loan. refer now. hurry! link: kredt.be/app"},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear customer enjoy republic day sale on amazon & flipkart with kreditbee zero cost emi. get extra 50% off on processing fee by using code janoff. offer valid only during jan 19th to 22nd.shop now, hurry! link: kredt.be/ecom'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'get processing fees discount worth rs. 150 on your first kreditbee loan. refer friends to register on kreditbee using your referral code nilcrewu8 and both you & your friend gets processing fees discount on your first kreditbee loan. hurry! link: kredt.be/refer'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'rs.50 paytm cashback on your first kreditbee loan. valid only for today. get loan now. hurry! link: https://kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'congratulations nikhil anand, avail upto rs. 450 discount on processing fee of your salary loan on kreditbee today. upload your salary docs & get verified to avail higher loans at lower interest rates. hurry! link: kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': "congratulations! you're upgraded again. kreditbee has unlocked your personal loan for salaried application. avail kreditbee personal loan for salaried now, pay later in emis. interest 24% p.a. onwards. hurry! link - https://kredt.be/app"},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'zero cost emis on kreditbee. shop with top sites like flipkart, amazon and more. complete your profile and explore now! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'rs.100 paytm cashback on your next kreditbee loan. valid till 10 feb. hurry! link - kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear ansifmon mytheenkutty, enjoy flexi personal loans and online purchase loans on kreditbee. avail now! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'congratulations!!your kreditbee profile has been reassessed.complete your application and avail an instant personal loan now! kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'avail instant loan upto 2 lakhs in 15 min. complete your kreditbee profile now & get loan directly into your bank account. hurry! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'approval',
                       'sms_template': 'dear mr./ms. aditya tara, pursuant to your loan application form submitted to kreditbee, a loan of the total amount of inr 15000 has been sanctioned by usha financial services private limited to you. please contact us on 08044292200 immediately in case of any disputes'},
                      {'loan_app': 'kreditbee',
                       'category': 'registration',
                       'sms_template': '191789 is otp for kreditbee. do not share otp with anyone.'},
                      {'loan_app': 'kreditbee',
                       'category': 'NACH',
                       'sms_template': 'your nach mandate registration request with kreditbee (finnovation tech sol pvt. ltd.) for your account ending xxxxxxxxxx6256 for loan repayments with adhoc frequency has been successfully completed. the mandate registration number is orbc0000000003740162. if you have not initiated the process of mandate registration, please call and report it to 08044292200'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'congratulations b sudhakar you have been pre-approved for a loan. keep your original address proof and pan cards ready for completing your profile. kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'congratulations saravanan, you have earned 30 points for using referral of raj and availing your first loan!'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear saravanan, 60 points are going to expire from your kreditbee points balance soon. use the points and avail a loan before they expire.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'free coupons worth rs. 4200 this big billion day on your first kreditbee online purchase emis. zero cost emis on flipkart, amazon. t&c apply. link: https://kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'congratulations! your credit limit has been increased on kreditbee. repay your existing loan and avail loan upto rs. 9000. hurry! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear kreditbee customer, please ensure that you make your installment payments on time to avoid penalties, reduction on bureau scores and to be eligible'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear alpesh mhatre, enjoy higher credit limit on kreditbee with personal loan for salaried. verify your salary profile now! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'flat rs. 100 paytm cashback on your first loan on kreditbee. money credited directly into bank account in 2 min. avail loan now. hurry! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear saravanan, 30 points have expired from your kreditbee points balance. current points balance is 30.'},
                      {'loan_app': 'kreditbee',
                       'category': 'reject',
                       'sms_template': 'dear nilesh, we could not approve your personal loan application at this moment. please re-apply in 6 months.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'exclusive offer: flat rs. 200 paytm cashback on availing kreditbee online purchase loans. enjoy zero cost emis & shop through flipkart, amazon. avail now! hurry! link: kredt.be/ecom'},
                      {'loan_app': 'kreditbee',
                       'category': 'apply',
                       'sms_template': 'congratulations rahul, your employment info is verified. please go ahead and complete your loan application to avail loan.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': "refer & earn with kreditbee. refer your friends to avail loan from kreditbee app, and get discount on your next kreditbee loan. get your referral code by scrolling to bottom of your home page screen, or go to 'my account' tab under refer & earn. link - http://nmc.sg/cautb5"},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'last step: verify your salary profile on kreditbee and remain ready for a salary loan. complete your profile now! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear uttam lal, enjoy flexi personal loans and online purchase loans on kreditbee. complete your profile and explore now! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear dixitkumar desai, rs.100 paytm cashback on your next kreditbee loan. direct bank transfer. avail loan now. valid till nov 30. t&c* link: kreditbee.in/dl?kb=pd'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'paperless ekyc is the future! win rs. 50 paytm cashback on updating your aadhar paperless ekyc on kreditbee and win an additional rs. 50 on your next loan. click on discover your credit & pricing to know more. offer valid till 20th nov. hurry! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear sujit kumar, one of your referred friends has got their profile approved. you have earned 50 points! current points balance is 50.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'offer- refer & earn unlimited cashbacks daily. help your friends avail evoucher loan on kreditbee using your code nikal7igc, and get rs.50 paytm cashback for each first kreditbee evoucher loan referred via you. offer valid till 31 aug. hurry! link - https://kredt.be/ecom_referal'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear rajbeer singh, your free credit score report requested through the kreditbee app, has been generated successfully! you can access the same by clicking on this link https://www.kreditbee.in/dl?kb=csr'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'limited period offer! get additional paytm cashback upto rs. 500 on your shopping during flipkart & amazon sale before 18th july. shop for mobile phones & tvs using kreditbee evouchers and pay back in easy emis. hurry! link - https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear javed thiyam, greetings for the day, we tried to reach your mobile number but there is no positive response. as we noticed that, your repayment of rs 2965 is pending and now its over-due 3 days and today is your last date for your payment. thus, request you to make the payment by today to avoid standard collection activities and further late payment charges.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'holi offer! get 100% off on pf on your e-voucher loan. why take a cash loan when you can save more. use code "holipf". link- kredt.be/ecom t&c apply'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'limited period offer! buy new phones from redmi, samsung & other electronic gadgets on flipkart sale using kreditbee emis. get additional paytm cashback upto rs.500 before 18th july. hurry! link - https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear shrinivas, please note that an auto debit request to repay your kreditbee(finnovation tech sol pvt. ltd.) loan with loan id 14151142 will be processed on your karnataka bank limited bank a/c xxxxxxxxxxxx2301 on 2020-03-30'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'exclusive: avail zero cost emis & 50% discount on processing fee (use buynow). coupons worth rs. 4200 free on kreditbee purchase loan. link: kredt.be/ecom'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear javed thiyam,despite several remainders, your repayment of rs 3186 is pending and now its over-due 16 days. hence, we urge you to make the payment at the earliest to avoid further late payment charges to maintain credit score and avoid getting supplementary legal action.'},
                      {'loan_app': 'kreditbee',
                       'category': 'reject',
                       'sms_template': 'sorry! we could not approve your loan application at this moment. please re-apply with correct details to continue. kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'avail instant salary advance upto rs. 1 lac in your bank account. your kreditbee profile is confirmed. travel now, pay later. http://nmc.sg/j6co9a'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'bbd sale ending today: use code buynow today and avail 50% discount on processing fee with kreditbee online purchase loans. zero cost emis on flipkart, amazon. additional coupons worth rs. 4200 free on your next online purchase loan. t&c apply. link: https://kredt.be/ecom'},
                      {'loan_app': 'kreditbee',
                       'category': 'approval',
                       'sms_template': 'dear mr./ms. sukhwinder singh, pursuant to your loan application form submitted to kreditbee, a loan of the total amount of inr 5000 has been sanctioned by usha financial services private limited to you. please contact us on 08044292200 immediately in case of any disputes'},
                      {'loan_app': 'kreditbee',
                       'category': 'apply',
                       'sms_template': 'dear ashish, we have received your kreditbee e-voucher loan request. please make the upfront payment for your loan after approval to receive your loan e-voucher. https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'last day today! get additional paytm cashback upto rs.500 on the upcoming flipkart sale days. avail easy e-vouchers on kreditbee app & pay later in easy emis. offer valid until 18th july only. hurry! apply now! link - https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': "last day today! get additional paytm cashback upto rs.500 on amazon's prime sale. buy any product using kreditbee evoucher & pay in low cost emis. hurry! link - https://kredt.be/app"},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear rukshad ansari, you are now eligible for a higher loan on kreditbee. get rs. 150 cashback on submitting your income proof. go to "my account" on the kreditbee app, select "discover your credit & pricing" & verify your salary now. hurry! link: kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'kreditbee diwali offer: use code salary and avail rs. 151 discount on processing fee on salary loan. win rs 4200 free coupons & bumper prize of samsung galaxy a30 for 51 winners. hurry! link: https://kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear ravi, we tried calling you but you were unreachable. avail e-vouchers on kreditbee and buy electronics, mobiles, book tickets on top ecommerce sites via kreditbee app. shop now, pay later in easy emis. hurry! link: https://kredt.be/smsecom'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear javed thiyam, despite several reminders, re- payment of rs 3033 is still not received and it is over- due by 7 days. we request you to clear your payments at the earliest to avoid adverse impact on your credit history and avoid any legal action being undertaken by kreditbee'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'avail exciting offers on kreditbee this diwali season. enjoy higher tenure and lower interest rates. direct bank acc transfer. avail loan now! kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'approval',
                       'sms_template': 'congratulations rahul, your application for higher credit limit loan has been approved. your new credit limit is rs 24000. hurry up and avail loan now.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'last sale of season: shop on flipkart, amazon, myntra sale with kreditbee purchase loans at zero cost emis. avail 50% discount on processing fee by using code buynow & win free coupons worth rs 4200. offer valid till sep 17. hurry! link: kredt.be/ecom'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear customer, get rs. 50 off on processing fee for you and your friends by referring them on kreditbee. refer now! link: kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'congratulations rahul! you are now eligible to avail a personal loan for salaried. avail an instant loan now! kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'dear customer enjoy republic day sale on amazon & flipkart with kreditbee zero cost emi. get extra 50% off on processing fee by using code janoff. offer valid only during jan 19th to 22nd.shop now, hurry! link: kredt.be/ecom'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': "limited period offer: free vouchers worth rs. 4200 on your first kreditbee e-voucher loan. click on 'shop' button & get kreditbee e-vouchers on flipkart, amazon, myntra, makemytrip and much more. pay later in easy emis. hurry! link: https://kredt.be/ecomac"},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'congratulations. you are eligible to apply for loans upto rs.2,00,000. set up an e-nach auto debit mandate and get your reference contacts verified to avail a higher credit line. https://kredt.be/app .'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'flipkart diwali sale: get 50% off on processing fee (use code buynow) on your next purchase loan on kreditbee. also, win free coupons worth rs.4200. shop for your favorite products at zero cost emis. offer valid till 16 oct. hurry! link: kredt.be/ecom'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'avail short term loans, salary loans and online purchase loans from kredibee and enjoy your festive season. avail loan now. hurry! link: https://kredt.be/home'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'get rs. 100 off on processing fee of your next kreditbee loan. refer your friends to avail a kreditbee loan after registering with your referral code & both of you win 100 points. your can get your referral code by clicking the refer button on the top right of the kreditbee app. hurry! link: kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': "pay '0% interest' on kreditbee e-voucher loan for shopping on flipkart, amazon, myntra & makemytrip. get trendy now & pay in easy emis later. hurry! link - https://kredt.be/home"},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear mohd mudassir, your emi rs.5834 has been delayed for 8 days we have not received any update on the payment. we request you to clear the pending emi at the earliest. on your failure to make payments towards the loan availed by you, kreditbee will be forced to the legal notice will be delivered at 17-1-178/40 darab jung colony,madannapet,hyderabad,500059 telangana. near masjid e salar e mulk ,hyderabad, telangana,500059, please make the payment today to avid the legal action, kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'promote',
                       'sms_template': 'you are one step away from taking a loan on kreditzy. get rs. 12000 directly in your bank in 5 mins. repayment starts @15 days. hurry! link: bit.ly/kzygen'},
                      {'loan_app': 'gotocash',
                       'category': 'registration',
                       'sms_template': 'dear customer, your login verification code for gotocash is 941886, valid for 30 minutes, please do not share or disclose.'},
                      {'loan_app': 'gotocash',
                       'category': 'registration',
                       'sms_template': 'dear customer, you are currently applying for a loan with gotocash, your verification code is 095175, valid for 30 minutes, please do not share or disclose.'},
                      {'loan_app': 'gotocash',
                       'category': 'apply',
                       'sms_template': 'dear customer, your loan application to gotocash has been submitted successfully, please wait for system review!'},
                      {'loan_app': 'gotocash',
                       'category': 'reject',
                       'sms_template': 'dear ramesh babu golla, we are sorry that your loan application to gotocash did not pass the eligibility requirements, you can apply again after 15 days, thank you.'},
                      {'loan_app': 'gotocash',
                       'category': 'overdue',
                       'sms_template': 'dear nikhil singh, the due date of your loan from gotocash is 3 days from now, total repayment amount is rs 5066.5, please login and repay. if you want to continue to use the loan, you can apply for an extension and extend the loan time.'},
                      {'loan_app': 'gotocash',
                       'category': 'overdue',
                       'sms_template': 'dear nishant shah, your loan from gotocash is overdue for 1 day. to avoid bad credit and other negative impacts, please pay immediately, total repayment is rs 5361.5.'},
                      {'loan_app': 'gotocash',
                       'category': 'reminder',
                       'sms_template': 'dear sobhitkumar naik, the due date of your loan from gotocash is today, total repayment amount is rs 6079.8. please login and repay, as failure to repay today will incur additional costs and affect your personal credit. if you want to continue to use the loan, you can apply for extension and extend the loan time.'},
                      {'loan_app': 'gotocash',
                       'category': 'approval',
                       'sms_template': 'dear sobhitkumar naik, your loan from gotocash has been disbursed to your account ending with 3931, amounting to 4598.16 rupees. please note and verify.'},
                      {'loan_app': 'gotocash',
                       'category': 'approval',
                       'sms_template': 'dear jayeshbhai jiniyabhai gamit , congratulations! your loan application to gotocash has been approved, we will transfer the funds to your bank account ending with 5665, please wait patiently.'},
                      {'loan_app': 'gotocash',
                       'category': 'repaid',
                       'sms_template': 'dear sobhitkumar naik, your loan from gotocash is paid off now. you could log in the app to apply for another loan.'},
                      {'loan_app': 'gotocash',
                       'category': 'overdue',
                       'sms_template': 'dear rajan kumar, due date extension on your loan from gotocash is successful, new due date is 01-04-2020. please pay on time to enjoy higher credit and other benefits.'},
                      {'loan_app': 'gotocash',
                       'category': 'overdue',
                       'sms_template': 'dear bhim singh, due date extension on your loan from gotocash is successful, new due date is 31-03-2020.please pay on time and keep good repayment record.'},
                      {'loan_app': 'gotocash',
                       'category': 'reject',
                       'sms_template': 'dear mohan raj, your application has been rejected due to deficient information. kindly provide additional documents to further process your application.'},
                      {'loan_app': 'gotocash',
                       'category': 'overdue',
                       'sms_template': 'dear badiya karthik, the due date of your loan from gotocash is 3 days from now, total repayment amount is rs 5066.5, please login and repay. if you wan'},
                      {'loan_app': 'cashbean',
                       'category': 'reminder',
                       'sms_template': 'your loan is due 07/02. pay on time to avoid late penalty.bit.ly/cashbeanrepay'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': "dear cashbean customer,ur loan is due. pay now to avoid penalty and low cibil score. if u're unreachable,w'll call your emergency contact bit.ly/cashbeanrepay"},
                      {'loan_app': 'cashbean',
                       'category': 'registration',
                       'sms_template': 'cashbn: 8098 is your otp, valid for 1 minute. cashbean never calls you asking for otp. please do not reveal it to anyone.'},
                      {'loan_app': 'cashbean',
                       'category': 'approval',
                       'sms_template': 'cashbean: money is transferred to your account, please repay on time and unlock higher loan amount for next time! click the link to take this survey: http://bit.ly/32o8q9h'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': 'your loan is due. pay now to avoid penalty and low cibil score. if you are unreachable, we will call your emergency contact from today. t.cn/eisapx1'},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': "need money on urgent basis?just fill your basic kyc info , you can win upto rs.10000 loan limit increase coupon. don't miss that out >>> bit.ly/cashbeanoffer"},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': 'your exclusive rewards have been sent to your account. expiring in 2 days. hurry! just try cashbean and check how many rewards you got. >>>bit.ly/cashbean1'},
                      {'loan_app': 'cashbean',
                       'category': 'repaid',
                       'sms_template': 'cashbean: your loan is now paid back.'},
                      {'loan_app': 'cashbean',
                       'category': 'announcement',
                       'sms_template': 'alert from cashbean: do not repay to yesbank(ifsc:yesb0cmsnoc)! pls only repay via cashbean app! if any issue, pls contact our customer service!'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': "dear cashbean customer,we've received ur account for higher collection activities. u may receive our calls soon.\n  to advoid click t.cn https://api.cashbean.in/t/mmebjz"},
                      {'loan_app': 'cashbean',
                       'category': 'reminder',
                       'sms_template': 'apka bhugtan {{repay_day}} se baki hai.late penalty se bachne ke liye samay pe bhuktaan kijiye, bit.ly/cashbeanrepay'},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': 'rs 20- 500 coupons have been put into your account. avail now and try cashbean! take your loan and get a chance to win rs 60,000. hurry up! >>>bit.ly/cashbean0'},
                      {'loan_app': 'cashbean',
                       'category': 'repaid',
                       'sms_template': 'cashbean: your loan is now paid back. relax and enjoy the better credit for future loans! looking forward to serving you again!'},
                      {'loan_app': 'cashbean',
                       'category': 'announcement',
                       'sms_template': 'hi customer,due to technical issues,bank transfer is not working now,pls repay using cashfree or paytm payment options only. we regret the inconvenience caused'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': 'ur acc. is due.we hvn\'t recvd any call frm u yet. u may term as "intentional defaulter",which may result in bad credit score t.cn https://api.cashbean.in/t/ziifzj .'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': "your loan is due .not paid,we send legal notice,get bad cibil,no company give u loan.if not contactable, we'll call ur emergency contact. t.cn/eisapx1"},
                      {'loan_app': 'cashbean',
                       'category': 'registration',
                       'sms_template': 'cashbn: 9992 is your verification code, valid for 10 minutes. please do not reveal it to anyone.'},
                      {'loan_app': 'cashbean',
                       'category': 'approval',
                       'sms_template': 'cashbean:your application is approved. please confirm and complete e-sign then get your money. valid for 2 day!'},
                      {'loan_app': 'cashbean',
                       'category': 'reminder',
                       'sms_template': 'your loan is due {{repay_day}}. pay on time to avoid late penalty. t.cn/eisapx1'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': "dear cashbean customer,we've received your details from higher collection team.kindly confirm if you still want to settle your loan account with us >https://api.cashbean.in/t/j7v632"},
                      {'loan_app': 'cashbean',
                       'category': 'reminder',
                       'sms_template': 'dear cashbean customer,ur1st instlmnt is due dys.rpymnt is 85%of totl loan+pnlty.if nt pay,cibil wil lw.if unreachabl,wil cl ur emrg cntct. pay with cashfree or paytm bit.ly/cashbeanrepay\\'},
                      {'loan_app': 'cashbean',
                       'category': 'reminder',
                       'sms_template': 'your 1st installment is due on{{day}}. repayment is 85% of total loan.pay on time to avoid late penalty. t.cn/eisapx1'},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': 'credit increasing reward has been put in your account.check this surprise right now >>> bit.ly/cashbeanoffer'},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': 'a super lucky coupon more than rs 300 has been put into your account. expiring in 1 day. try cashbean now & check how much you can get. >>>bit.ly/cashbean0'},
                      {'loan_app': 'cashbean',
                       'category': 'repaid',
                       'sms_template': 'cashbean: your loan is now paid back. relax and enjoy flip to win for this holi >>> http://bit.ly/cashbeanholi'},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': "super! you're selected to get rs 30 to 1000 cashbean rewards, expiring only for 2 days! find it on [my account]>[coupon]>>>bit.ly/cashbean0"},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': 'your friend rahul just took rs 35,000 loan from cashbean. exclusive coupon from rs. 20 to 200 has been put in your account. expiring in 1 day. just try cashbean and check how much you can get. >>>bit.ly/cashbean0'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': 'your loan is due.if unable to pay,cibil score will low,no loan company give you loan.if you are unreachable,will call your emergency contact. t.cn/eisapx1'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': 'your loan is overdue.if no pay,cibil score will be low and no loan company give you loan.if you are unreachable,will call your emergency contact bit.ly/cbloan'},
                      {'loan_app': 'cashbean',
                       'category': 'reminder',
                       'sms_template': 'dear cashbean customer,\n  as we found that you are not paying your due of cashbean. so we are giving you offer. if you will your cashbean payment before 12 am midnight then you may get upto 100% cashback. kindly make your payment right now to this offer. and if you will not pay your dues tonight before 12 am then we will take legal action against you. so kindly make your payment right now to get this offer and also avoid legal action against you.\n  regarding,\n  cashbean'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': "your loan is overdue. repay with cashfree or paytm(bank transfer is not working) if you're unreachable, we will call ur emergency contact. t.cn/eisapx1"},
                      {'loan_app': 'cashbean',
                       'category': 'reminder',
                       'sms_template': 'make it merry with cashbean! pay your loan now and get a chance to join spin & win for christmas gifts! get a chance to win iphone 11 pro, cash prize up to rs 60,000!>>>bit.ly/cashbeanch'},
                      {'loan_app': 'cashbean',
                       'category': 'reminder',
                       'sms_template': 'your loan is due 10/03 kindly use cashfree or paytm options to pay(bank transfer is not working) to avoid late penalty. t.cn/eisapx1'},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': 'happy new year 2020! rs 20- 500 coupons have been put into your account. avail now and try cashbean! take your loan now and get the chance to win rs 60,000 cash. hurry up! >>>bit.ly/cashbean0'},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': 'happy new year 2020! a super lucky coupon more than rs 300 has been put into your account. expiring in 1 day. try cashbean now & check how much you can get. >>>bit.ly/cashbean0'},
                      {'loan_app': 'cashbean',
                       'category': 'reminder',
                       'sms_template': 'we wish you a happy diwali this year. your loan is due on {{repay_day}}. pay on time to avoid late penalty. t.cn/eisapx1'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': 'your loan is overdue.we send legal notice if unpaid,get bad cibil and no corp give you loan.if not reachable,we will call your emergency contact bit.ly/cbloan'},
                      {'loan_app': 'cashbean',
                       'category': 'overdue',
                       'sms_template': "dear customer,\n  today is your last chance for making cashbean loan repayment . you are 30 days late.last day of your payment.\n  final call !! your account may forwarded to legal department for further action.\n  we are providing another chance to pay your outstanding loan amount. you have 2 hours to make payment.\n  pay your loan outstanding loan within 2 hours. if we don't received your payment, legal action may be taken against you. \n  to avoid further activities. call/sms/what's app -(7428384353/7428384363)\n  pay immediatly.\n  \n  regards\n  cashbean\n  collection team"},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': "super! you're selected to get rs 30 to 1000 cashbean rewards, expiring only for 2 days! find it on [my account]>[coupon]>>>bit.ly/cashbean0"},
                      {'loan_app': 'cashbean',
                       'category': 'promote',
                       'sms_template': 'your exclusive rewards have been sent to your account. expiring in 2 days. hurry! just try cashbean and check how many rewards you got. >>>bit.ly/cashbean1'},
                      {'loan_app': 'icredit',
                       'category': 'registration',
                       'sms_template': '162547 is your icredit otp.for security,please do not share it with anyone.'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[ icredit ] my valued customer, you have not make the [moneyclick] payment of your loan within the due date. request you to make payment immediately, failure of which will result in initiation of legal proceedings'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[ icredit ] alert! overdue for [moneyclick] will have charges. it will affect your cibil credit records in lifetime and it is a serious situation! you must repay now!'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[ icredit ] new bonus: open the app to apply for partial deferral, the platform will stop launch collection service to you for 7 days as to ease your financial pressure!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': 'dear customer, now it is the time for your repayment. pls repay in time and you will get a chance to increase your credit amount after repayment.'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[icredit]dear customer,we have not received outstanding amt of 8916.3 which was due on 26/03/2020.pay immediately to avoid penalties and litigation bit.ly/2or9sxh'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] dear customer, icredit app remind you, today is your repayment date. start your prepayment now. remember not to have overdue prepayment!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] we have not received any response yet from your repayment. repay earlier and you can get chances to increase your credit amount by repaying'},
                      {'loan_app': 'icredit',
                       'category': 'approval',
                       'sms_template': '[ icredit ] congratulations! your application has been approved. open icredit app for withdrawal cash.'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] good morning! clearing your [moneyclick] repayment to have a nice day! you are eligible to apply next loan after repayment. remember do not overdue repayment'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] dear customer! we have not received any response yet from your repayment. you can apply for next loan after repayment'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': "[ icredit ] we didn't receive your [moneyclick] repayment. increase the credit amount after repayment.please repay in time!"},
                      {'loan_app': 'icredit',
                       'category': 'repaid',
                       'sms_template': '[ rupeeplus ] congratulations, your loan product of [moneyclick] has been paid off! welcome to apply for your next loan.'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': 'dear client we still not received the amount 3828 that was due on 28/03/2020 your payment should be received today end of the day,we will not take legal action. we urge you not to ignore this last reminder. https://play.google.com/store/apps/details?id=com.jc.icredit'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] after a few hours, your [moneyclick] loan order will cause overdue fees and credit smudges, pls repay now !!!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] after a few hours, your loan order will generate overdue fees and credit smudges, please repay now !!! remember not to have overdue repayment!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] tomorrow (repayment_date) is the last repayment date! please repay on time as not to affect your personal credit. by the way, you can continue to apply for a loan after repayment.'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[icredit] today is the repayment date, you can apply for the next loan after repayment, your credit limit will be higher, remember not to repay overdue! repayment information is subject to app. do not transfer funds to any private account.'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] pay off your [moneyclick] loan, no overdue fees and get more credit amount. overdue repayment will bring you more overdue fees!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] alert! your [moneyclick] loan repayment is late for 2 day. overdue will bring you more and more overdue fees. you must repay now!'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[icredit] your loan slip is overdue by 1 day. please repay immediately to avoid affecting your personal credit. by the way, repayment information is subject to app. do not transfer funds to any private account.'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[ icredit ] alert! overdue of [moneyclick] will cause fees. your cibil credit records will be affected in lifetime and it is a serious situation! pls'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] dear customer, good evening, today is your repayment due date! please complete your repayment today to avoid overdue fees!'},
                      {'loan_app': 'icredit',
                       'category': 'approval',
                       'sms_template': '[ rupeeplus ] dear customer, congratulations! your application of [moneyclick] loan product is ready to withdraw! open rupeeplus and get money. just in 1 minute!'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[ icredit ] alert! overdue will bring you more and more overdue fees. pls repay now! [ moneyclick ]'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[ icredit ] alert! late [moneyclick] repayment will be recorded in your national credit report. overdue is not good. repay now!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': 'dear customer, avoid overdue charges and affect credit score, please pay on time. btw, the earlier you repay, more credit amount you get!'},
                      {'loan_app': 'icredit',
                       'category': 'approval',
                       'sms_template': '[icredit] your application has been processed and approved. open app to withdraw cash. system will automatically transfer loan amount to your bank account'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[icredit] dear customer, you have not paid off your loan within the maturity date. you are required to pay immediately, otherwise legal action will result. remember， repayment information depends on the app. do not transfer funds to any private account.'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[moneyclick] warning! overdue will bring you more and more overdue fees. after repayment you can apply next loan. pls start repayment now!repayment information is subject to app. do not transfer funds to any private account.'},
                      {'loan_app': 'icredit',
                       'category': 'reject',
                       'sms_template': "[ moneyclick app ] sorry your application hasn't been approved! we look forward to your re-application."},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[ icredit ] warning! overdue will have charges. it will affect your cibil credit records in lifetime and it is a serious situation! overdue repayment will bring you more overdue fees!you must repay now!'},
                      {'loan_app': 'icredit',
                       'category': 'overdue',
                       'sms_template': '[ icredit ] warning! you have been overdue for 2 days. this will affect your credit limit, which is a serious situation! you must repay now! repayment information is subject to app. do not transfer funds to any private account.'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[icredit] today is your repayment date, please repay immediately, serious overdue will lead to legal proceedings on the platform, remember not to overdue! repayment information is subject to app. do not transfer funds to any private account.'},
                      {'loan_app': 'icredit',
                       'category': 'promote',
                       'sms_template': '[ icredit ] dear customer, good news! your credit amount from [moneyclick] loan product has been updated. open icredit app to withdraw, just in 1 minute!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[icredit] good morning! today is your repayment day, i wish you a good day to pay off your repayment! you can apply for the next loan after repayment. remember not to overpay, the repayment information is subject to app. do not transfer funds to any private account.'},
                      {'loan_app': 'icredit',
                       'category': 'approval',
                       'sms_template': '[ icredit ] dear customer! your application has been approved, please open app to withdraw cash in time!'},
                      {'loan_app': 'icredit',
                       'category': 'approval',
                       'sms_template': '[ icredit ] your withdraw is timeout! but we will reserve your credit amount. pls apply again. open app to apply and withdraw!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': 'dear customer we would like to remind you that the 2904 was due for payment on 28/03/2020to avoid further cost please repay the payment no later than due date. https://play.google.com/store/apps/details?id=com.caft.rupeeplus'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] hello, today is your repayment day, please pay immediately, if the repayment is successful, your credit limit will be doubled!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] dear customer, good noon, tomorrow is your last repayment day, please remember to repay, otherwise there will be overdue fees!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] please open the moneyclick app and pay immediately. repayment in advance will greatly increase your credit limit and you can also apply for the next order! remember not to overpay.repayment information is subject to app. do not transfer funds to any private account.'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick]dear customer, please make sure to repay today, or you will incur overdue fees and reduce your credit limit!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] dear customers, please pay off your loan within the due date today. failure to do so will lead to legal proceedings'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] today is your repayment day! please start paying off immediately! overdue will seriously affect your credit limit!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] good news! repayment one day in advance can increase the credit limit and more chances to double the credit limit! seize the opportunity and repay now! participate!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] based on your credit, the platform invites you to repay in advance! if you start paying now, you will have the opportunity to upgrade your credit line!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick]dear customer, the repayment countdown is 1 day, please repay immediately! overdue will seriously affect your credit limit!repayment information is subject to app. do not transfer funds to any private account.'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] dear customer, good evening, please repay in advance! now you can increase your credit limit if you repay successfully. after repayment, you can continue to apply for the next loan!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[icredit] dear customer, 10 rupee coupons have been put into your account, today is your last repayment day, please go to repay, remember not to overdue!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] there are three days left to repay. please repay in advance! repayment in advance has the opportunity to increase the credit limit!repayment information is subject to app. do not transfer funds to any private account.'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': "[ moneyclick ] today is your last repayment day. the credit line is directly related to your repayment. don't overdue or you will be affected!"},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] dear customer, good afternoon, please pay off in advance! now, if you make a successful repayment, you can increase your credit line. after repayment, you can continue to apply for the next loan!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] dear valued customer, you have not make the payment of your loan within the due date. request you to please make the payment immediately, failure of which will result in initiation of legal proceedings. so, kindly pay your loan amount by just one click on bit.ly/2or9sxh so that there is no negative impact on your credit score and moreover no penalties or legal costs are charged in future'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': 'the last payment date for your due amount 9648 is 30/11/2019. please complete the payment before this date to avoid penalties.\n https://play.google.com/store/apps/details?id=com.jc.icredit'},
                      {'loan_app': 'icredit',
                       'category': 'approval',
                       'sms_template': '[icredit] dear customer, you have passed the risk control review and are currently in the \n process of making loans. please pay attention!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] warning! overdue will have charges. pay off your loan, no overdue fees and no credit stains. you must repay now!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[moneyclick] dear customer, there is only 1 day left to repay. please go to the app to repay now!'},
                      {'loan_app': 'icredit',
                       'category': 'promote',
                       'sms_template': '[icredit] hi, in order to increase your credit limit, we invite you to apply for a loan \n \n again!'},
                      {'loan_app': 'icredit',
                       'category': 'promote',
                       'sms_template': '[icredit] good news!as long as you apply for a loan again,you will have the opportunity to get cash discount coupons at different prices!'},
                      {'loan_app': 'icredit',
                       'category': 'reminder',
                       'sms_template': '[ icredit ] after a few hours, your loan order will generate overdue fees and credit smudges, please repay now !!! repay earlier and you can get chances to increase your credit amount by repaying your loan in time! if you pay off, please ignore it.'},
                      {'loan_app': 'icredit',
                       'category': 'promote',
                       'sms_template': '[icredit] dear customer, since you are in good standing, you are now welcome to apply for a loan again, the offer is waiting for you!'},
                      {'loan_app': 'cashmama',
                       'category': 'registration',
                       'sms_template': 'your verification code for login at cashmama app is: 673363. valid for 10 minutes.'},
                      {'loan_app': 'cashmama',
                       'category': 'disbursal',
                       'sms_template': '(receipt reminder) dear mukesh kumar, the loan has been disbursed to your bank with account number [6469], repay on time to avoid late payment charges.'},
                      {'loan_app': 'cashmama',
                       'category': 'reject',
                       'sms_template': 'dear mohammed nazaf alishaikh, your loan application has not passed the review, please login to the app again on 14-04-2020.'},
                      {'loan_app': 'cashmama',
                       'category': 'repaid',
                       'sms_template': '(successful repayment) dear mukesh kumar, we have successfully received payment of rs. 3040.00. please wait for 24 hrs to reflect the amount in your loan.'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': 'dear saurabh pawar,this is warm reminder that 3days later is your bill day . please make repayment in time to avoid late payment fees and affect your credit.'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': '(expiration reminder) dear pradeep pandit, your rs. 3000.00 loan is due today, please repay in time to avoid overdue penalty.'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': '(repayment reminder) dear darshil acharya, your loan which loan amount is rs 5000.00 is due on 28-03-2020, repay on time to avoid late payment charges.'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': '(repayment reminder) dear pradeep pandit, your loan of amount rs. 3000.00 is due on 08-02-2020, repay on time to avoid late payment charges.'},
                      {'loan_app': 'cashmama',
                       'category': 'approval',
                       'sms_template': 'dear sandeep singh, the loan application for rs 3000.00 has been approved. the loan will be paid to your within 24 hours, please note to check.'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': '(expiration reminder) dear mahendra kumar amanta, your rs. 5000.00 loan is due today, please repay in time to avoid overdue penalty.'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': '(repayment reminder) dear mahendra kumar amanta, your loan of amount rs. 5000.00 is due on 17-02-2020, repay on time to avoid late payment charges.'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': '(expiration reminder) dear preeti sharma, your loan which loan amount is rs 5000.00 is due today, please repay in time to avoid overdue penalty.'},
                      {'loan_app': 'cashmama',
                       'category': 'approval',
                       'sms_template': 'dear mukesh kumar, the loan application which loan amount is rs 5000.00 has been approved. the loan will be paid to your within 24 hours, please note to check.'},
                      {'loan_app': 'cashmama',
                       'category': 'approval',
                       'sms_template': 'dear mahendra kumar amanta, the loan application for rs 5000.00 has been approved. the loan will be paid to your within 24 hours, please note to check.'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': '(repayment reminder) dear praveen kumar bhartiya, your loan which loan amount is rs 3000.00 is due on 31-03-2020, repay on time to avoid late payment charges.'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': '(expiration reminder) dear sandeep singh, your loan which loan amount is rs 3000.00 is due today, please repay in time to avoid overdue penalty.'},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': '(overdue reminder) dear balaji r, your loan of rs. 3000.00. it is currently overdue for 1 day. please repay in time to avoid overdue penalty.'},
                      {'loan_app': 'cashmama',
                       'category': 'approval',
                       'sms_template': 'dear sandeep singh, the loan application which loan amount is rs 3000.00 has been approved. the loan will be paid to your within 24 hours, please note to check.'},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': '(overdue reminder) dear kannadasan m, your loan which loan amount is rs 5000.00, it is currently overdue for 3 day. please repay in time.'},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': '(overdue reminder) dear savta mali, your loan which loan amount is rs 3000.00, it is currently overdue for 3 day. please repay in time.'},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': 'final intimation from cash mama dear customer, your account is now seriously past due. we have sent you numerous payment requests and still have not received your past due payment. ensure that your cibil score is not impacted by paying today. this will also help you to get more credit on cashmama platform based on your good payment behavior. >final warning< make the payment by today . if not actions will be taken on your e-sign documents. legal notice may be issued to registered address and bank and work address you will have to face legal consequences along with fine and court charges. queries 9618025566'},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': '**warning* : *legal notice issued to registered address and bank and you will have to face legal consequences along with legal charges.* \ndear customer;\nthis is to bring to your notice that despite numerous requests for payment from our end, your amount still remains due for payment.\nwe demand the payment of your outstanding amount immediately otherwise company will be blocking your all loans pathways \n3 years you will not be getting any loans from the financial institutions.\nyour credit score will be affecting.\nyou will be the blacklisted customer.\nyou will have to face some serious legal issues.\nbecause notice is ready\nso requesting you to clear your and resolve the problems\nteam cashmama legal department queries:9618025566'},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': "we have delivered repeated warnings to you regarding your long overdue. due to your acquiescence in this matter, and continued failure to pay, we have referred this matter to an agent to continue proceedings. \ntherefore, we request you to please make the payment immediately or else , we will add your profile to blacklist and you won't able to take any further loans up to 3 years from financial institution this will be your last opportunity to avoid further legal proceedings and the substantial costs associated there with. team cashmama legal department"},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': "final warning cashmama legal department:-\nwe have delivered repeated warnings to you regarding your long overdue. due to your acquiescence in this matter,  and continued failure to pay, we have referred this matter to an agent to continue proceedings.  \ntherefore, we request you to please make the payment immediately  or else , we will add your profile to blacklist and you won't able to take any further loans  from financial institution this will be your last opportunity to avoid further legal proceedings and the substantial costs associated there with.\n from cashmama legal department\nbefore we take further actions we would like to inform your all contacts.\ncontact - 7680061122"},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': 'team cashmama legal department warning legal notice issued to registered address and bank and you will have to face legal consequences along with fine and court charges. this is to inform you that you have still not made the payment so we have already sent multiple correspondences regarding the same, but still now we have not got a single response from your end by paying the amount therefore we request you to make the payment immediately or else , we will add your profile to blacklist and if cibil score updated , after 7 years you can get credit facility from any bank. as your record of default will not stay in cibil for more than 7 years. this will be your last opportunity to avoid further legal proceedings and the substantial costs associated there with. our company has assigned person to you they will come to your home address with notice be at your home today once you got the legal notice you have to go court and pay the amount with legal charges use otp vueeydujxh (9618025566)'},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': "you have taken loan from cash mama app we have delivered repeated warnings to you regarding your long overdue. due to your acquiescence in this matter,  and continued failure to pay, we have referred this matter to an agent to continue proceedings.  \nfrom the time you reading this, you will have to settle this matter with us and make payment. this will be your last opportunity to avoid further legal proceedings and the substantial costs associated therewith.before we take further actions we would like to inform your contacts.before we take further actions we would like to inform your contacts.\nwe will send notice to your contact's that you are a default customer  queries 9618025566"},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': 'dear cashmama customer,for such reporting. as the next steps, we would like to bring to your notice that*cashmama* will be may initiating an fir/legal case against you towards the fraudulent activity based on your non-payment at your sole risk and cost. please note the company has sufficient evidence to support its claim against you. non-payment of dues will result in legal action kindly disregard this message if payment of the amount due in cashmama has already been processed. if not, you can make the payment using *cashmama*application today if not a case number to be generated at location station and intimated to you via call and messages once case number generated you can check in the link below here for your case details, https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/case_no.php?state=d&state_cd=3&dist_cd=20 check this link clearly need to pay the amount today if you are ignoring this we will proceed this file contact number:9618025566'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': 'repayment reminder: your loan has expired, please repay as soon as possible. once it is overdue, there will be penalty interest and affect the cibil score, making it difficult to borrow from all institutions and banks.'},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': 'overdue reminder: your loan is overdue for  day, please repay as soon as possible. serious overdue will have high penalty interest and affect cibil scores, making it difficult to borrow from all institutions and banks.'},
                      {'loan_app': 'cashmama',
                       'category': 'reminder',
                       'sms_template': 'dear avasarala venkatrao, please repay your loan in cashmama, you can pay quickly through the following payment link: https://p-y.tm/wtb-kxi'},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': "* team cashmama* * warning **legal notice may be issued to registered address  and work address. you will have to face legal consequences along with fine and court charges. dear customer, this is to inform you that you have still not made the payment so we have already sent multiple correspondences regarding the same,we request you to please make the payment by  today.\n* if not ,we may report to cibil,highmark, equifaux as loan defaulter. \n*you won't able to take any further loans from any financial  institute in future.\n* we may initiate legal notice to your registered address and aadhaar address. once you get the notice you have to go court and pay the amount with court charges. \nqueries:8978777326 teamcashmama"},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': '*team cashmama legal department* we are initiating your file for legal proceedings. it will affect below mentioned aspects * this will directly affect your cibil score * if cibil score is less than 750 you will not be able to get any loan or credit card. * once u will get notice you will have to go to legal court and pay with legal charges according to (ipc) section 420 and in the indian penal code deals with cheating and dishonestly there is still time. however, to avoid suit and legal advisory action. you need to make the payment of due amount within *2 hours* time. this will be your last opportunity to resolve the matters without the expense of court proceedings and to avoid the legal advisory action against you. cibil to be zero until the case is solved.warning to non responding customers company is taking strict action and will forwarding all the documents adhaar and pan details to nearest police station of your address for any queries    9963799787'},
                      {'loan_app': 'cashmama',
                       'category': 'overdue',
                       'sms_template': 'cashmama legal department warning default customer we are taking strict action,we have started legal proceeding against your f.i.r is filed at nearest local police station of your residence.you will receive a legal court notice shortly to your aadhar address we will informed to your workplace/collage of your behavior and this will reflect on your character and present address which you have provided to us and will forwading all the documents adhaar and pan card to civil court notice issued on the basis of rbi guidelines kindly remember that this is a criminal offence related to the under section 138e act of 1881 loan being an agreement or understanding between the bank and the borrower the general laws like law of contract,you have not cleared your overdueamount,we have a case against you because of breach of contract ban on adhara card and pan card till the case is resolved.company to freeze your bank accounts.to speak legal head mahesh takur wavier of on penalty (8374463529)'},
                      {'loan_app': 'cashmama',
                       'category': 'approval',
                       'sms_template': 'dear bavya b s, the loan application for amount 5000.00 has been approved. we will transfer it to your account, it may take 6-24hrs on bank working days, thank you for using cashmama.'},
                      {'loan_app': 'rupeemax',
                       'category': 'registration',
                       'sms_template': '(rupeemax) 2226 is your verification code. this code will expire in 10 minutes. please do not disclose it for security purpose.'},
                      {'loan_app': 'rupeemax',
                       'category': 'reminder',
                       'sms_template': '(rupeemax) dear client, your loan due date is coming. please pay on time. for more instant personal loans, click https://bit.ly/2kk1dgu'},
                      {'loan_app': 'rupeemax',
                       'category': 'approval',
                       'sms_template': '(rupeemax) congratulations! dear sir/madam, your loan application has been approved! please login the app to sign the electronic contract and confirm the loan details.'},
                      {'loan_app': 'rupeemax',
                       'category': 'registration',
                       'sms_template': 'dear user 4049 is your one time verification code,this code will be expired in 10 minutes.'},
                      {'loan_app': 'rupeemax',
                       'category': 'reminder',
                       'sms_template': '(rupeemax)dear client, your loan will due in 1 day.kindly pay on time to keep good credit.you can download all upgrade apps through: https://bit.ly/2psu2sn'},
                      {'loan_app': 'rupeemax',
                       'category': 'reminder',
                       'sms_template': '(rupeemax) dear client, today is the due date. please pay immediately. for more instant personal loans, click https://bit.ly/2kk1dgu'},
                      {'loan_app': 'rupeemax',
                       'category': 'reminder',
                       'sms_template': '(rupeemax)dear client,today is the due date.please pay immediately.you can download all upgrade apps through: https://bit.ly/2psu2sn'},
                      {'loan_app': 'rupeemax',
                       'category': 'reminder',
                       'sms_template': '(rupeemax) dear client, your loan due date is coming. please pay on time. for more instant personal loans, click https://bit.ly/2tadlck'},
                      {'loan_app': 'rupeemax',
                       'category': 'overdue',
                       'sms_template': '(rupeemax)dear client,your loan has been overdued for 3 days.please pay now as your credit goes negative.you need download upgrade app: https://bit.ly/2psu2sn'},
                      {'loan_app': 'rupeemax',
                       'category': 'overdue',
                       'sms_template': '(rupeemax) dear sir/madam, your loan has been in overdue state for 2 days. please pay immediately as your credit score is going in negative state.'},
                      {'loan_app': 'rupeemax',
                       'category': 'overdue',
                       'sms_template': '(rupeemax) dear client, your loan has been overdue. please pay immediately. for more instant personal loans, click https://bit.ly/2tadlck'},
                      {'loan_app': 'rupeemax',
                       'category': 'reminder',
                       'sms_template': '(rupeemax) dear sir/madam, your loan due date is coming. please pay on time to boost your credit score.'},
                      {'loan_app': 'rupeemax',
                       'category': 'reminder',
                       'sms_template': '(rupeemax) dear sir/madam, your loan has expired. please pay immediately to avoid negative credit score and further penalties.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear hemant mundekar, despite several reminders, re-payment of rs 2732 is still not received and it is over-due by 9 days. request you to make the due payment immediately, failing which your credit history will be impacted and we will be forced to initiate legal action against you.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear shivakumar g shiva, your account is past due with rs. 5731. please make the payment by today to avoid late payment fees & your secured credit future, for payments you can use your kreditbee app, net banking, google pay, phonepe, paytm & bank transfer. kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear kishore rentapalli, considering the current covid 19 crisis, we would like to extend our support to you in these challenging times. as a privileged customer, we are waving off your full penalty on your due. please pay your emi amount today. offer valid till today. for more details contact on 9945927358. kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear kishore rentapalli, considering the current covid 19 crisis, we would like to extend our support to you in these challenging times. as a privileged customer, we are waving off your full penalty on your due. please pay your emi amount today. offer valid till today. kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear kishore rentapalli, reminder: pay back the emi today to ensure that 3 days delay dpd is not updated to cibil. a bad cibil score with default status will be a threat to your future as you will not be eligible for loans in any financial sector. kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': "dear kishore rentapalli, reminder: payback before 1st april'20 to ensure that a 6 days delay dpd is not updated to cibil. a bad cibil score with default status will be a threat to your future as you will not be eligible for loans. kreditbee team."},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear kishore rentapalli, we pray that you all and your family stay healthy, happy and safe in this epidemic situation. you all are really important so the company giving you a 100% discount on your overdue charges use this opportunity to make payment through google pay, phonepe or any other net banking. kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'ಆತ್ಮೀಯ shivakumar g shiva, ನಿಮ್ಮ ಖಾತೆಯು ರೂ. 5724. ಪಾವತಿಗಳಿಗಾಗಿ ನಿಮ್ಮ ಕ್ರೆಡಿಟ್ ಬೀ ಅಪ್ಲಿಕೇಶನ್, ನೆಟ್ ಬ್ಯಾಂಕಿಂಗ್, ಗೂಗಲ್ ಪೇ, ಫೋನ್ ಪೇ, ಪೇಟಿಎಂ ಮತ್ತು ಬ್ಯಾಂಕ್ ವರ್ಗಾವಣೆಯನ್ನು ಬಳಸಬಹುದು. ಕ್ರೆಡಿಟ್ ಬೀ ತಂಡ.'},
                      {'loan_app': 'kreditbee',
                       'category': 'approval',
                       'sms_template': 'congratulations syedmusthafa! your kreditbee personal loan for salaried application with credit limit rs. 16000 is approved. avail loan in your bank a/c in 2 mins. hurry! https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'reminder',
                       'sms_template': 'dear yogesh patne, this is a reminder message from kreditbee regarding your emi amount. request you to kindly make the payment within time to avoid the penalty charges, failing might impact your credit history. kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'last day reminder: pay your emi today to ensure that your cibil score is not updated with  days delay dpd. ensure that your financial future is not negatively impacted and pay now. kreditbee legal team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'this is a friendly reminder that your emi is still remains unpaid for  days overdue. we would appreciate immediate payment. please get in touch with us for further assistance. kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'alert -  please clear your repayment today to avoid any legal action. legal formalities will be processed. avoid your name and documents before moving to defaulter lists. next time you would not be able to take any loan from any other banks or financial organizations. kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'ಆತ್ಮೀಯ nikhil j j, ಕೊನೆಯ ದಿನ ಜ್ಞಾಪನೆ: ನಿಮ್ಮ ಸಿಬಿಲ್ ಸ್ಕೋರ್ ಅನ್ನು  ದಿನಗಳ ವಿಳಂಬ ಡಿಪಿಡಿಯೊಂದಿಗೆ ನವೀಕರಿಸಲಾಗಿಲ್ಲ ಎಂದು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳಲು ಇಂದು ನಿಮ್ಮ ಇಎಂಐ ಪಾವತಿಸಿ. ನಿಮ್ಮ ಆರ್ಥಿಕ ಭವಿಷ್ಯದ ಋಣಾತ್ಮಕ ಪರಿಣಾಮ ಇಲ್ಲ ಮತ್ತು ಈಗ ಪಾವತಿಸುವ ಎಂದು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳಿ. ಕ್ರೆಡಿಟ್ ಬೀ ಕಾನೂನು ತಂಡ.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'we urge you to pay your total due amount soon to ensure that a  days delay dpd is not updated to cibil. a bad cibil score with default status means that your profile will be blacklisted with all the banks and credit card companies. kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'overdue',
                       'sms_template': 'dear kredit bee customer we pray that you all and your family stay healthy,happy and safe in this epidemic situation.you  all are really important so company giving you 100%  discount on your over due charges use this opportunity make payment through google pay. phone pay. any net banking for more information call to 9164314401, kredit bee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'reducefee',
                       'sms_template': 'dear makkapati bharathi, considering the current covid 19 crisis, we would like to support by waving off 100% penalty on your installment. please pay your emi amount. for more details call on 6366564071. kreditbee team.'},
                      {'loan_app': 'kreditbee',
                       'category': 'extension',
                       'sms_template': 'dear customer, you are eligible to get upto 2 months moratorium on your kreditbee loan. either postpone your loan or pay due amount, else the loan will be marked as delayed. your cibil score may get impacted in this case. https://kredt.be/app.'},
                      {'loan_app': 'kreditbee',
                       'category': 'extension',
                       'sms_template': 'dear customer, we would like to support you by increasing the interest waiver period till 13th april. pay by 13th april to enjoy the benefits and get priority treatment for your future loans.'},
                      {'loan_app': 'kreditbee',
                       'category': 'extension',
                       'sms_template': 'dear chhagan, your kreditbee loan kb200201kdwut of rs. 5126 has been rescheduled successfully as per your request. the new schedule has been sent to your registered email address and is available on the kreditbee app as well. https://kredt.be/app'},
                      {'loan_app': 'kreditbee',
                       'category': 'extension',
                       'sms_template': '<#> 911525 is otp for postponing instalment. do not share code with anyone vueeydujxhz'},
                      {'loan_app': 'nanocred',
                       'category': 'registration',
                       'sms_template': 'nanocd: 6894 is your otp, valid for 1 minute. nanocred never calls you asking for otp. please do not reveal it to anyone.'},
                      {'loan_app': 'nanocred',
                       'category': 'disbursal',
                       'sms_template': 'nanocred: money is transferred to your account, please repay on time and unlock higher loan amount for next time!'},
                      {'loan_app': 'nanocred',
                       'category': 'approval',
                       'sms_template': 'nanocred:your application is approved. please confirm and complete e-sign then get your money. valid for 2 day!'},
                      {'loan_app': 'nanocred',
                       'category': 'repaid',
                       'sms_template': 'nanocred: your loan is now paid back. relax and enjoy the better credit for future loans! looking forward to serving you again!'},
                      {'loan_app': 'nanocred',
                       'category': 'announcement',
                       'sms_template': 'hi customer,due to technical issues,bank transfer is not working now,pls repay using paytm payment options only. we regret the inconvenience caused'},
                      {'loan_app': 'nanocred',
                       'category': 'announcement',
                       'sms_template': 'alert from nanocred: do not repay to yesbank(ifsc:yesb0cmsnoc)! pls only repay via nanocred app! if any issue, pls contact our customer service!'},
                      {'loan_app': 'nanocred',
                       'category': 'reminder',
                       'sms_template': 'dear nanocred customer, ur loan is due on 26/03. pls repay with cashfree(bank transfer is not working now) to avoid late penalty. \n  bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'reminder',
                       'sms_template': 'your loan is due 21/02. pay on time to avoid late penalty. bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'promote',
                       'sms_template': 'need money on urgent basis?just fill your basic kyc info , you can win upto rs.10000 loan limit increase coupon. don’t miss that out >>> bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'apply',
                       'sms_template': 'nanocred: we cannot proceed your application today. please re-submit later (no need to fill information again). thank you!'},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': 'dear nanocred customer,ur loan is due on15/03.repay with cashfree to avoid bad cibil score. if not reachable,we will call your contacts bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'reminder',
                       'sms_template': 'your loan is due on 11/03. pls repay with cashfree(bank transfer is not working now) to avoid late penalty. bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': 'dear nanocred customer,ur loan is due on10/03.repay with cashfree to avoid bad cibil score. if not reachable,we will call your contacts bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': 'your loan is due. pay now to avoid penalty and low cibil score. if you are unreachable, we will call your emergency contact from today. bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': 'your loan is due.if unable to pay,cibil score will low,no loan company give you loan.if you are unreachable,will call your emergency contact. bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': 'dear nanocred customer,ur loan is due on22/03.pay now to avoid bad cibil score. if not reachable,we will call your contacts bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'reject',
                       'sms_template': 'sorry, your order review failed due to your telephone is not answered'},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': 'today is your last day to make your payment. tomorrow we may initiate a legal notice and we will consider that you have not paid the outstanding willingly. t.cn https://api.cashbean.in/t/yfjjzv .'},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': "your loan is due .not paid,we send legal notice,get bad cibil,no company give u loan.if not contactable, we'll call ur emergency contact. bit.ly/nanocred"},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': 'important notice! since your outstanding loan account has still not paid, we are on the verge of taking legal action againts you. pay imediately https://api.nanocred.in/t/yyvyiz .'},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': 'dear nanocred customer,loan is due on 19/02. legal notice will be sent to you on non-payment of loan.if not repay will call your contacts.repay with cashfree  bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'reminder',
                       'sms_template': 'dear nanocred customer,your payment for your loan is due on 31/03. pay on time to avoid late penaltybit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': 'ur loan is due on 31/03. pay now to avoid penalty and low cibil score. if you are unreachable, we will call your emergency contact from today. bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'overdue',
                       'sms_template': 'ur loan is due on 31/03. if unable to pay,cibil score will low,no loan company give you loan.if you are unreachable,will call your emergency contact. bit.ly/nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'reminder',
                       'sms_template': 'you can do a bank transfer to nanocred using these details , account number: 2223330021076170, ifsc code: ratn0vaapis, beneficiary name: nanocred'},
                      {'loan_app': 'nanocred',
                       'category': 'approval',
                       'sms_template': 'dear customer, failed to disburse to your bank account, please change your bank info on app or contact us for help, thank you!'},
                      {'loan_app': 'kissht',
                       'category': 'login',
                       'sms_template': '<#> your kissht otp for login: 018913\n ppoelbuowxz'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'dear mukesh, congratulations! you are eligible for the fastcash revolving line of credit of rs 4000 for 2 years. kindly submit your bank details to proceed.'},
                      {'loan_app': 'kissht',
                       'category': 'disbursal',
                       'sms_template': 'dear mukesh, your line of credit is approved and the amount has been disbursed in your bank account successfully. it may take few hours to update.'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'hello, your fastcash instalment of rs. 4463.00 is due on 21-01-2020. pay on time to avoid late fees & maintain your credit score. pay now https://tuc95.app.goo.gl/noce9'},
                      {'loan_app': 'kissht',
                       'category': 'announcement',
                       'sms_template': "dear customer, repay your fastcash using online link '- https://tuc95.app.goo.gl/hthny , or upi id '- 777019529562804.03@cmsidfc , or neft/imps bank details '- beneficiary name: si creva capital services pvt ltd, bank: idfc bank, a/c no: 777019529562804, a/c type: current account, ifsc: idfb0010209, branch: prabhadevi branch"},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'dear chirag khatva, greetings from kissht. kindly send emi payment of rs 5098.4 on our upi id 777019574182485.03@cmsidfc or you can use our bank details for neft transfer, name si creva capital services pvt ltd, bank idfc bank, ac no 777019574182485, ifsc idfb0010209, prabhadevi branch. need help? 022-48913044'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'dear nilesh deore, we have received a total payment of rs 4529.95. a detailed bifurcation has been sent to you on mail. any payment issues? reach us at 022 62820570'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'dear haroon, kindly use the below link to pay the outstanding amount on your kissht loan. click here https://kis.ht/u9eg'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'dear customer, nothing makes us happier than a loyal customer! give us a 5-star rating on google play store to improve our services and credit line for you. click here - https://d5p5q.app.goo.gl/fmgd'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'repay kissht loan of rs.6763.3 using upi id- 777017069362764.03@cmsidfc or do neft- name: sicreva capital, idfc bank, a/c: 777017069362764, ifsc: idfb0010209'},
                      {'loan_app': 'kissht',
                       'category': 'repaid',
                       'sms_template': 'dear abdul quddus, we have received a total payment of rs 6452.71. detailed bifurcation is sent on your registered email. thank you!'},
                      {'loan_app': 'kissht',
                       'category': 'repaid',
                       'sms_template': 'dear mohammad bohari, we have received a total payment of rs 6850.00. a detailed bifurcation has been sent to you on mail. any payment issues? reach us at 022 62820570'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'hello, your fastcash instalment of rs. 2677.00 is due today. pay now to avoid late fees & maintain your credit score. pay https://tuc95.app.goo.gl/pudcm'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'hi achla! your opinions matter and keep us motivated! rate us 5-star on playstore using https://d5p5q.app.goo.gl/fmgd'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'dear niranjan khare, you can now avail a special offer to extend the due date of fastcash by 15 days. just pay rs. 960.52 & save on late payment penalties. hurry! offer expires tonight 11:59 pm. pay using https://tuc95.app.goo.gl/uupkk tca'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'your kissht loan of rs.4856.00 is due on 26-02-2020. pay here https://kis.ht/hsi7 to avoid late fees & bad credit score'},
                      {'loan_app': 'kissht',
                       'category': 'approval',
                       'sms_template': 'dear bhawani, congratulations! you are eligible for an amount of rs 4850 with kissht. kindly submit your bank details to proceed.'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'hey! win rs.50 paytm cash. rate us on google playstore https://d5p5q.app.goo.gl/fmgd & write how fastcash helped you meet an emergency cash need. top 10 best stories of the day to win. t&c'},
                      {'loan_app': 'kissht',
                       'category': 'disbursal',
                       'sms_template': 'dear pooja, congrats! your loan application is approved and the amount has been transferred into your bank account successfully.'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'hello, your kissht loan re-payment of rs.4500 is due on 29/11/2019. pay on-time to avoid late fees & maintain your credit score. pay now https://tuc95.app.goo.gl/jyaxw'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'your kissht loan of rs.4886.00 is due on 03-04-2020. pay here https://kis.ht/cp4n to avoid late fees & bad credit score'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'payment overdue! dear chirag, you have not made the fastcash payment of rs. 4886.00 on the due date. pay now using https://tuc95.app.goo.gl/hv1jy to avoid paying hefty late payment fees. need help? reach us at 022 62820570'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'hello, your kissht loan re-payment of rs.3526.00 is due on 26-11-2019. pay on-time to avoid late fees & maintain your credit score. pay now https://tuc95.app.goo.gl/noce9'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'your kissht loan of rs.4886.00 is due today. pay here https://kis.ht/bmqt to avoid late fees & bad credit score'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'dear rahul singh, you can now avail a special offer to extend the due date of fastcash by 21 days. just pay rs. 1818.62 & save on late payment penalties. hurry! offer expires tonight 11:59 pm. pay using https://kis.ht/f1es tca'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'payment outstanding - avoid late fee charges! dear amar, your fastcash payment is overdue by 2 days. your total outstanding is rs. 3022. pay now using https://tuc95.app.goo.gl/xkzve to avoid penalty.'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'its now or never - increased credit! congratulations! your fast cash line of credit is increased to rs.8200. available this offer and get credit in your account within 2 mins. hurry! limited period offer. get it now\n  https://d5p5q.app.goo.gl/ptdq'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'legal court case reminder - dear ajit, you have failed to make the payment of your kissht loan outstanding amount. we, therefore, urge you to not ignore this reminder & make payment immediately on https://tuc95.app.goo.gl/ju6cb. failing which we will be constrained to take legal actions under section 420 (cheating & dishonesty) & section 405 (criminal breach of trust) of the indian penal code, 1860. to stop legal action, call us now at 02262820553'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'avoid late fee charges! dear md, your fastcash payment is overdue by 3 days. a very high late payment fee will be applied. pay outstanding using 4886 within 24 hours to avoid paying penalty fees!'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': "avoid legal action!\n \n dear muhammed, you are now 4 days past your payment due date with kissht. we will initiate legal action against you in the next few days and file a police complaint if you don't make the payment of your outstanding amount of rs. 6380.3 immediately. your account has also been temporarily suspended.\n pay now using https://tuc95.app.goo.gl/njkjb to activate account and avoid legal actions.need help? reach us at 022 62820570"},
                      {'loan_app': 'kissht',
                       'category': 'login',
                       'sms_template': '190533 is your login otp. treat this as confidential. sharing it with anyone gives them full access to your kissht account.'},
                      {'loan_app': 'kissht',
                       'category': 'repaid',
                       'sms_template': 'dear abdul mannan, we have received a total payment of rs 7707.00. detailed bifurcation is sent on your registered email. thank you!'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': "don't delay paying outstanding kissht loan that may affect your credit health and put your score below 600. pay now weurl.co/ncl0qa"},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'legal action initiated!\n \n dear krishna chaitanya, we have initiated legal action and will also report your non-payment of outstanding payment to all credit bureaus in india. this will impact your score and affect your chances to receive any loans in future. your current outstanding amount is now rs. 5433.\n pay now using https://tuc95.app.goo.gl/2bxbo in 24 hours to avoid serious impact to your credit score.'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'non payment of the kissht loan will result in credit score reduction. pay your kissht outstanding now! weurl.co/mcb6vt'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'hello, your fastcash instalment of rs. 6427.00 is due on 21-02-2020. pay on time to avoid late fees & maintain your credit score. pay now https://kis.ht/twrq'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'कानूनी अदालत के मामले का स्मरणपत्र - प्रिय jetendra, आप अपने kissht ऋण बकाया राशि का भुगतान करने में विफल रहे हैं। आपसे अनुरोध करते हैं कि इस रिमाइंडर को अनदेखा न कर इस लिंक पर https://tuc95.app.goo.gl/4fbrd तुरंत भुगतान करें। भुगतान न होने पर हमारी टीम भारतीय दंड संहिता, 1860 के धारा 420 (धोखाधड़ी और बेईमानी) और धारा 405 (आपराधिक विश्वासघात) के तहत कानूनी कार्रवाई करने के लिए विवश होगी। कानूनी कार्रवाई को रोकने के लिए, 08061915543 पर अभी कॉल करें'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'snooze offer extended for 24 hrs - delay fastcash repayment by paying a small fee & stop your delinquency from being reported to cibil. pay here https://tuc95.app.goo.gl/wq88a or click snooze payment on kissht app. for details, call 08061915543. hurry! offer expires tonight 11:59 pm. tca'},
                      {'loan_app': 'kissht',
                       'category': 'login',
                       'sms_template': '578753 is your otp for changing mobile number. treat this as confidential. sharing it with anyone gives them full access to your kissht account.'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'emi of rs.1234.00 for kissht loan id 1164378 is due on 05-02-2020. kindly keep your account funded to avoid charges. pay now -https://tuc95.app.goo.gl/j3qn5'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'hello, thank you for choosing kissht! rate us 5-star on playstore to improve services & credit line for you. rate now - https://d5p5q.app.goo.gl/fmgd'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'payment overdue! your fastcash payment of rs.4070.00 was due yesterday. to avoid late fees pay immediately https://kis.ht/8eeo'},
                      {'loan_app': 'kissht',
                       'category': 'reject',
                       'sms_template': 'dear perfectionist prince, thank you for applying for loan with kissht. your application has been denied. your transaction id is 12590503.'},
                      {'loan_app': 'kissht',
                       'category': 'apply',
                       'sms_template': 'hi, your document income proof has been requested rework for the transaction id : fran15749403864058, due to incorrect doc. kindly reupload the document from the below link https://tuc95.app.goo.gl/mg5hu'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'its now or never!! save your credit score by repaying your existing kissht loan. pay now weurl.co/mc6afd to save it.'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'आपका सिबिल स्कोर 1 दिन में कम हो जाएगा - आपका नया लोन डिफ़ॉल्ट डेटा अगले 24 घंटों में cibil को दिया जायेगा। इससे आपके सिबिल स्कोर में बहुत बड़ी कमी आएगी। हमारी टीम तो आपकी डिफ़ॉल्ट रिपोर्ट सिबिल को देने से रोकने के लिए, 08061915543 पर अभी कॉल करें, अन्यथा पेमेंट करने के लिए https://tuc95.app.goo.gl/zvmph क्लिक करें'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'आप भाग्यशाली हैं ! आपकी fastcash line of credit को बढ़ाकर rs. 3000 कर दिया गया है। अपने बैंक खाते में 2 मिनट में नकद प्राप्त करने के लिए इस ऑफ़र का लाभ उठाएं। जल्दी कीजिये! ऑफर सीमित अवधि के लिए। अभी प्राप्त करने के लिए https://d5p5q.app.goo.gl/ptdq पर क्लिक करें |'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'avoid late fees! your fastcash payment of rs.5913 is overdue by 2 days. late fees to be charged soon. pay now https://kis.ht/hmhf'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'legal court case - dear pramod, our team is filing legal case against you in local court under section 420 (cheating & dishonesty) & section 405 (criminal breach of trust) of the indian penal code, 1860. you will soon receive a summon at your registered address. you still have a chance to stop legal court case filing, by repaying your kissht loan outstanding. pay now https://kis.ht/mvue or call 08061915543 or click here to chat https://kis.ht/wa'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': '24 hrs left - last reminder - your fastcash payment is overdue by 3 days. late fees will be charged tonight. pay now https://kis.ht/hsi7'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': "it's your lucky day today!\n congratulations! you are now eligible for fastcash line of credit with higher credit limit.get credit in your account within 2 mins.hurry! limited period offer. apply now https://d5p5q.app.goo.gl/ptdq"},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'your payment of ₹3859.9 is overdue for the last 4 days. further delay in payment will reflect negatively on your credit score! pay now weurl.co/oc0h9j'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'today is the last day to avoid late fees on your overdue payment of ₹5624.2. pay now!! weurl.co/nc26op'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'hello, your kissht loan re-payment of rs.4030.00 is due today. pay now to avoid late fees & maintain your credit score. pay https://tuc95.app.goo.gl/xkv4t'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'नमस्कार, हमें उम्मीद है की आपको kissht की fastcash सुविधा पसंद आयी. गूगल प्लेस्टोरे पर kissht app को करें, ताकि अन्य ग्राहक भी आपके अनुभव का लाभ ले सकें. यहां रेट करें https://d5p5q.app.goo.gl/fmgd'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'now close your kissht outstanding loan of rs.153680 /- by paying just rs.115260 /-without any penalty charges. apply https://kis.ht/zr9q to avail this offer now!'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': "dear customer, repay your fastcash using online link '- - , or upi id '- 777017293159701.03@cmsidfc , or neft/imps bank details '- beneficiary name: si creva capital services pvt ltd, bank: idfc bank, a/c no: 777017293159701, a/c type: current account, ifsc: idfb0010209, branch: prabhadevi branch"},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'are you tired of frequent collection calls & messages? get extra time to repay your loan - just pay a small amount to stop all collection calls. select snooze payment on kissht app or click https://kis.ht/gzvv to avail this offer. offer expires tonight 11:59pm. for details call 08061915543 or click here to chat https://kis.ht/wa'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'hello, your kissht loan re-payment of rs.3000 is due today. pay now to avoid late fees & maintain your credit score. pay https://tuc95.app.goo.gl/kuury'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'ప్రియమైన కస్టమర్, నమ్మకమైన కస్టమర్ ను పొందడం కంటే మారే మీ మాకు సంతోషాన్ని ఇవ్వదు! మా సేవలను మీకోసం మెరుగుపరచడానికి గూగుల్ ప్లేస్టోర్ లో మాకు 5 - స్టార్ రేటింగ్ ఇవ్వండి. ఇక్కడ క్లిక్ చేయండి https://d5p5q.app.goo.gl/fmgd'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': "it's now or never - avail fastcash!\n congratulations! we have now activated fastcash line of credit for you.avail this offer and get credit in your account within 2 mins.hurry! limited period offer. apply now https://d5p5q.app.goo.gl/ptdq"},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': '3 days left - last chance to extend your fastcash repayment period by 15 days. just pay rs.951.08 and you will not receive any collection reminder calls & will not be charged any further late payment fees for next 15 days. pay now using https://tuc95.app.goo.gl/hx97a offer expires on 2020-01-09 23:59. tca'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'your cibil score will fall tomorrow- your latest loan default data will be reported to cibil in next 24 hrs. this will lead to big reduction in your cibil score. to stop us from reporting your default to cibil, call us now at 08061915543. click to pay https://tuc95.app.goo.gl/1dez8'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'why play with your credit score health? pay your kissht loan outstanding and save your credit score. pay now weurl.co/ncav52'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'hey! potential to win rs.200 paytm cash. rate us on social media or google playstore https://d5p5q.app.goo.gl/fmgd & write how fastcash helped you meet an emergency cash need. top 10 best stories of the day to win. winners will be announced on kissht facebook page by 2nd march -t&c apply'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'அன்புள்ள durairaj, உங்கள் கிஷ்ட் கடன் நிலுவைத் தொகையை செலுத்தத் தவறிவிட்டீர்கள். எனவே, இந்த நினைவூட்டலை புறக்கணிக்க வேண்டாம் மற்றும் இந்த இணைப்பை https://tuc95.app.goo.gl/2xdtb பயன்படுத்தி உடனடியாக பணம் செலுத்துமாறு கேட்டுக்கொள்கிறோம். செலுத்தவில்லை என்றால், 1860 ஆம் ஆண்டு இந்திய பெனல் குறியீட்டின் பிரிவு 420 (மோசடி மற்றும் நேர்மையின்மை) மற்றும் பிரிவு 405 (குற்றவியல் நம்பிக்கை மீறல்) சட்டங்கள் கீழ் உங்கள் மீது நடவடிக்கை எடுக்கப்படும்.சட்ட நடவடிக்கை நிறுத்த, இப்போதே எங்களை அழைக்கவும் 02262820553.'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'your case has been given for field collection - dear customer, we have submitted your case to our field team for collection. our team can visit your home anytime and enquire with your neighbours and family in your absence. request you to co-operate with them. to stop our collection team from visiting your home, pay now https://kis.ht/exs1 or call 08061915543 or click here to chat https://kis.ht/wa'},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'hello, your fastcash instalment of rs. 4070.00 is due on 03-01-2020. pay on-time to avoid late fees & maintain your credit score. pay now https://tuc95.app.goo.gl/8wppn'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'snooze ऑफर का आखरी दिन - अब आप एक छोटे से शुल्क का भुगतान करके अपने fastcash चुकौती में देरी कर सकते हैं, और साथ ही अपने गैर-भुगतान को cibil में रिपोर्ट होने से रोकें। भुगतान यहां करें https://tuc95.app.goo.gl/inpjd अथवा kissht app पर snooze payment विक्लप का चयन करें। जानकारी के लिए, 08061915543 पर कॉल करें। यह ऑफर आज रात 11:59 बजे समाप्त होगा। शर्तें लागू।'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'snooze offer - delay fastcash repayment by paying a small fee & stop your delinquency from being reported to cibil. pay here https://tuc95.app.goo.gl/vb71m or click snooze payment on kissht app. for details, call 08061915543. hurry! offer expires soon. tca'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'only 24 hrs left - last chance to extend your fastcash repayment period by 15 days. just pay rs.951.08 and you will not receive any collection reminder calls & will not be charged any further late payment fees for next 15 days. pay now using https://tuc95.app.goo.gl/staja hurry! offer expires on 2019-12-22 23:59. tca'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': "have you checked your reduced credit score? it will be reduced further if you don't pay kissht loan. pay now weurl.co/ncajxr"},
                      {'loan_app': 'kissht',
                       'category': 'reminder',
                       'sms_template': 'dear dinesh chaudhri, greetings from kissht. kindly send emi payment of rs 4266 on our upi id 777019359959557.03@cmsidfc or you can use our bank details for neft transfer, name si creva capital services pvt ltd, bank idfc bank, ac no 777019359959557, ifsc idfb0010209, prabhadevi branch. need help? 022-48913044'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'your cibil score will fall in 2 days- your latest loan default data will be reported to cibil in next 48 hrs. this will lead to big reduction in your cibil score. to stop us from reporting your default to cibil, call us now at 02262820553. click to pay https://tuc95.app.goo.gl/tr7lk'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'dear dinesh  chaudhri, you can now avail a special offer to extend the due date of fastcash by 15 days. just pay rs. 951.08 & save on late payment penalties. hurry! offer expires tonight 11:59 pm. pay using https://tuc95.app.goo.gl/akdyf tca'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'legal court case reminder - dear dinesh, you have failed to make the payment of your kissht loan outstanding amount. we, therefore, urge you to not ignore this reminder & make payment immediately on https://kis.ht/rbvr. failing which we will be constrained to take legal actions under section 420 (cheating & dishonesty) & section 405 (criminal breach of trust) of the indian penal code, 1860. to stop legal action, call us now at 08061915543 or click here to chat https://kis.ht/wa'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': "don't worry, kissht is with you in all situations. get emi extension of 21 days with 0 late fees. just pay ₹951.08 weurl.co/ocdync expiry 31-03-20 tca"},
                      {'loan_app': 'kissht',
                       'category': 'login',
                       'sms_template': '<#>  your kissht otp for login: 900450\nppoelbuowxz'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'last chance – only 7 hours left to avail pay 20% offer to postpone for 21 days weurl.co/rclrqc'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'be wise-timely payment increases your credit score & late payment damages it. let your timely payment help us give you more loans. pay now weurl.co/scbjve'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'your credit score is fair but it can be excellent. repaying the existing kissht loan will just do that. pay now weurl.co/nco98b'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'credit score damage- we have submitted your data to all credit bureaus. every single day of delay will now impact & reduce your score. pay now weurl.co/qcxn4r to save your score'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'lockdown making kissht payment difficult? just pay only 20% weurl.co/qckw72 & postpone payment for 21 days. 100% dues waiver. valid for 24 hrs.'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'in these troubled times, damaging your credit score, by not paying just rs.5422.4, can make things worse. pay now weurl.co/qce1ow & save your credit score'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'offer valid for 24 hrs- need time to repay? pay just ₹1146.02 & get 21 days with 0 interest, 100% late fee waiver & improved credit record. pay weurl.co/scexo6'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'legal court case – dear parikshit, our team is filing legal case against you in local court under section 420 (cheating & dishonesty) & section 405 (criminal breach of trust) of the indian penal code, 1860. you will soon receive a summon at your registered address. you still have a chance to stop legal court case filing, by repaying your kissht loan outstanding. pay now weurl.co/hcxzk0 or call 08061915543 or click here to chat weurl.co/hcxzi0'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'snooze offer - delay fastcash repayment by paying a small fee & stop your delinquency from being reported to cibil. pay here https://tuc.app.goo.gl/roxa or click snooze payment on kissht app. for details, call . hurry! offer expires soon. tca'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'snooze offer last day - delay fastcash repayment by paying a small fee & stop your delinquency from being reported to cibil. pay here https://tuc.app.goo.gl/roxa or click snooze payment on kissht app. for details, call . hurry! offer expires tonight : pm. tca'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'snooze offer extended for  hrs - delay fastcash repayment by paying a small fee & stop your delinquency from being reported to cibil. pay here https://tuc.app.goo.gl/roxa or click snooze payment on kissht app. for details, call . hurry! offer expires tonight : pm. tca'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'our case has been given for field collection - dear customer, we have submitted your case to our field team for collection. our team can visit your home anytime and enquire with your neighbours and family in your absence. request you to co-operate with them. to stop our collection team from visiting your home, pay now https://tuc.app.goo.gl/po or call'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'are you tired of frequent collection calls & messages? get extra time to repay your loan - just pay a small amount to stop all collection calls. select snooze payment on kissht app or click https://kis.ht/2zfw to avail this offer. offer expires tonight 11:59pm. for details call 08061915543.'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': '24 ಗಂಟೆಗಳವರೆಗೆ ವಿಸ್ತರಿಸಿದ ಸ್ನೂಜ್ ಆಫರ್ - ಸಣ್ಣ ಶುಲ್ಕವನ್ನು ಪಾವತಿಸುವ ಮೂಲಕ ಫಾಸ್ಟ್\u200cಕ್ಯಾಶ್ ಮರುಪಾವತಿಯನ್ನು ತಡಮಾಡಿ ಮತ್ತು ನಿಮ್ಮ ಅಪರಾಧವನ್ನು ಸಿಬಿಲ್\u200cಗೆ ವರದಿ ಮಾಡುವುದನ್ನು ನಿಲ್ಲಿಸಿ. ಇಲ್ಲಿ ಪಾವತಿಸಿ https://tuc95.app.goo.gl/gj3ss ಅಥವಾ ಕಿಸ್ತ್ ಅಪ್ಲಿಕೇಶನ್\u200cನಲ್ಲಿ ಸ್ನೂಜ್ ಪಾವತಿ ಕ್ಲಿಕ್ ಮಾಡಿ. ವಿವರಗಳಿಗಾಗಿ, 02262822680 ಗೆ ಕರೆ ಮಾಡಿ. ಬೇಗಮಾಡಿ ! ಕೊಡುಗೆ ಇಂದು ರಾತ್ರಿ 11:59 ಕ್ಕೆ ಮುಕ್ತಾಯಗೊಳ್ಳುತ್ತದೆ. ತೆಗೆದುಕೊಳ್ಳಿ ಮತ್ತು ಕಾಳಜಿ ವಹಿಸಿ'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'dear mahammed  ali, you can now avail a special offer to extend the due date of fastcash by 15 days. just pay rs. 1153.1 & save on late payment penalties. hurry! offer expires tonight 11:59 pm. pay using https://tuc95.app.goo.gl/bfbds tca'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'filing of police case - dear mahammed, you have ignored all our previous payment reminder & not yet paid your kissht loan outstanding. we are filing a police case against you under section 420 (cheating & dishonesty) & section 405 (criminal breach of trust) of the indian penal code, 1860. to stop further legal proceedings, pay now https://kis.ht/fjbz or call 08061915543 or click here to chat https://kis.ht/wa'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'your case has been given for field collection - dear customer, we have submitted your case to our field team for collection. our team can visit your home anytime and enquire with your neighbours and family in your absence. request you to co-operate with them. to stop our collection team from visiting your home, pay now https://kis.ht/fjbz or call 08061915543 or click here to chat https://kis.ht/wa'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'save your cibil - you are 4 days past your due date. late payment will lead to big reduction in your cibil score. pay now https://kis.ht/3pzq'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'cibil score at risk - your non-payment of kissht loan is being reported to cibil. to stop cibil damage, pay now https://kis.ht/3pzq'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'important clarification - dear customer, request you to not delay your emi payment, based on recent news/announcements in wake of the covid-19 lockdown, as this can impact your credit score & future loan eligibility. pay your kissht loan outstanding of rs.8059 now using kissht app or click https://kis.ht/zrbg to maintain a healthy credit score.'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'save your credit score - you are 4 days past your due date. late payment will lead to big reduction in your credit score. pay now https://kis.ht/3pzq'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'credit score at risk - your non-payment of kissht loan is being reported to credit bureaus. to stop, pay now https://kis.ht/3pzq'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'important-customer, we are filing legal case against you in court under sec 420 & 405 of indian penal code, to stop legal case, pay now weurl.co/rcdzhe'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'legal reminder - dear chauhan, further delay in repayment of your kissht loan can result in legal actions against you under sec 420 (cheating & dishonesty) & sec 405 (criminal breach of trust) of the indian penal code, 1860. pay now https://kis.ht/wvr1 to stop legal action. chat here https://kis.ht/wa'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'pay rs.1516.77 for extra time to repay your loan & save credit score. click https://kis.ht/dmwx or click snooze payment on kissht app chat https://kis.ht/wa tca'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'offer expires @ 11:59 pm- pay rs.1516.77 for extra time to repay loan. click https://kis.ht/dmwx or click snooze payment on kissht app. chat https://kis.ht/wa tca'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'last chance to stop legal action- pay just rs.1516.78 & get 21 days relief from legal action, 100% late fee waiver & 0 interest. pay weurl.co/pch37c offer expires at 12 midnight'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'few hours left- pay just rs.1516.78 & get 21 days relief from legal action, 100% late fee waiver & 0 interest. pay weurl.co/pcm5vj offer expires at 12 midnight'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'important- dear chauhan, we are filing legal case against you in local court under sec 420 (cheating & dishonesty) & sec 405 (criminal breach of trust) of the indian penal code, 1860. you will soon receive summon at your registered address. to stop legal case, pay now weurl.co/qcwpgq'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'you repaid your past kissht loans on time, but not paid your latest outstanding. pay now https://kis.ht/wvr1 to avail more loans. chat https://kis.ht/wa tca'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'snooze offer - delay fastcash repayment by paying just rs. 1759.38 & stop your delinquency from being reported to cibil. pay here https://kis.ht/mcyk or click snooze payment on kissht app. for details, call 02262822680 / chat https://kis.ht/wa kindly ignore if paid. offer expires soon. tca'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'last day of snooze - delay fastcash repayment by paying just rs. 1759.38 & stop your delinquency from being reported to cibil. pay here https://kis.ht/mcyk or click snooze payment on kissht app. call 02262822680 / chat https://kis.ht/wa kindly ignore if paid. offer expires tonight 11:59 pm. tca'},
                      {'loan_app': 'kissht',
                       'category': 'extension',
                       'sms_template': 'snooze extended for 24 hrs - delay fastcash repayment by paying just rs.1759.38 & stop your delinquency from being reported to cibil. pay here https://kis.ht/mcyk or click snooze payment on kissht app. call 02262822680 / chat https://kis.ht/wa kindly ignore if paid. offer expires tonight 11:59 pm. tca'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'legal court case reminder – dear mohiddin, you have failed to make the payment of your kissht loan outstanding amount. we, therefore, urge you to not ignore this reminder & make payment immediately on weurl.co/gcdw6u. failing which we will be constrained to take legal actions under section 420 (cheating & dishonesty) & section 405 (criminal breach of trust) of the indian penal code, 1860. to stop legal action, call us now at 08061915543 or click here to chat weurl.co/gcdwbv'},
                      {'loan_app': 'kissht',
                       'category': 'reject',
                       'sms_template': 'document verification!\nhi ganesh, verification of the initial documents uploaded by you was not successful. click here weurl.co/mc0r5u to check the details and upload the required documents. need help? reach us at 02262820570'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'increase your credit score to above 700 by repaying your existing kissht loan. further delay will lead to higher reduction in credit score. pay now https://kis.ht/52pn or call 08061915543 or click here to chat https://kis.ht/wa'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'field collection will meet you- dear customer, we have submitted your case to our field team for collection. our team can visit your home anytime & enquire with your neighbours & family in your absence. kindly co-operate with them. to stop collection home visit,pay now weurl.co/lcf5q1 or to chat weurl.co/lcf5x1'},
                      {'loan_app': 'kissht',
                       'category': 'overdue',
                       'sms_template': 'tired of collection calls? just pay ₹0 & stop all such calls. click weurl.co/ncbalq or click snooze payment on kissht app chat weurl.co/ncbamq tca'},
                      {'loan_app': 'kissht',
                       'category': 'promote',
                       'sms_template': 'remember the top 10 customer stories contest? the winners have been announced! click here now weurl.co/ecfrle to check if you are one of them!'},
                      {'loan_app': 'cashbus',
                       'category': 'registration',
                       'sms_template': '745029 is your cashbus otp.for security,please do not share it with anyone.'},
                      {'loan_app': 'cashbus',
                       'category': 'reminder',
                       'sms_template': 'cashbus: you loan is due on 04/02. pay on time to avoid late penalty. http://j.mp/32jqz0u'},
                      {'loan_app': 'cashbus',
                       'category': 'approval',
                       'sms_template': 'cashbus: you application is approved. please confirm and complete e-sign then get you money. valid for 2 days!'},
                      {'loan_app': 'cashbus',
                       'category': 'disbursal',
                       'sms_template': 'cashbus: money is transferred to you account, please repay on time and unlock higher loan amount for next time!'},
                      {'loan_app': 'cashbus',
                       'category': 'reminder',
                       'sms_template': 'cashbus?you loan is due on 03/25. pay on time to avoid late penalty . http://j.mp/32jqz0u'},
                      {'loan_app': 'cashbus',
                       'category': 'promote',
                       'sms_template': "cashbus: hey! you're selected to increase your loan limit! a surprise offer has been put in your account. check how much you can get.>>> http://j.mp/32jqz0u"},
                      {'loan_app': 'cashbus',
                       'category': 'reminder',
                       'sms_template': 'cashbus: you loan is due on 04/04. pay on time to avoid late penalty. http://j.mp/32jqz0u'},
                      {'loan_app': 'cashbus',
                       'category': 'reminder',
                       'sms_template': 'cashbus: you loan is due on 02/13. but to avoid forgetting to pay back tomorrow, you can pay today. continue to enjoy the next service. http://j.mp/32jqz0u'},
                      {'loan_app': 'cashbus',
                       'category': 'overdue',
                       'sms_template': 'your loan has been overdue and a liquidated damages have been incurred. pay now to avoid penalty and low cibil score.http://j.mp/32jqz0u'},
                      {'loan_app': 'cashbus',
                       'category': 'reminder',
                       'sms_template': 'cashbus: you loan is due on 03/31. but to avoid forgetting to pay back tomorrow, you can pay today. continue to enjoy the next service. http://j.mp/32jqz0u'},
                      {'loan_app': 'cashbus',
                       'category': 'reminder',
                       'sms_template': 'cashbus: you loan is due on 04/04. but to avoid forgetting to pay back tomorrow, you can pay today. continue to enjoy the next service. http://j.mp/32jqz0u'},
                      {'loan_app': 'cashbus',
                       'category': 'registration',
                       'sms_template': '[cashtrain] 5148 (cashtrain sms verification code),valid within 30 minutes.'},
                      {'loan_app': 'cashbus',
                       'category': 'reminder',
                       'sms_template': "cashbus?you loan is due on 02/08. today is your last repayment time.if you can't pay back on time today. you will have to pay an extra fine.pay on time to avoid late penalty. http://j.mp/32jqz0u"},
                      {'loan_app': 'cashbus',
                       'category': 'reminder',
                       'sms_template': 'cashbus: you loan is due on 01/31.your loan is due.pay now to avoid penalty and low cibil score.if you are unreachable,we will call your emergency contact from today. http://j.mp/32jqz0u'},
                      {'loan_app': 'cashbus',
                       'category': 'overdue',
                       'sms_template': 'cashbus: your loan has been overdue for one day and a penalty has been incurred. pay now to avoid fines and lower cibil scores. http://j.mp/32jqz0u'},
                      {'loan_app': 'cashbus',
                       'category': 'repaid',
                       'sms_template': 'cashbus: your loan is now paid back. relax and enjoy the better credit for future loans! looking forward to serving you again!'},
                      {'loan_app': 'cashbus',
                       'category': 'approval',
                       'sms_template': 'cashbus: dear customer, unfortunately, your loan has not been signed on time and already been expired, please re-submit.'},
                      {'loan_app': 'paysense',
                       'category': 'registration',
                       'sms_template': '<#> 916296 is your paysense verification code and will be valid for the next 15 minutes\n j1xi/0k6opy'},
                      {'loan_app': 'paysense',
                       'category': 'reject',
                       'sms_template': 'dear preeti, we regret to inform you that your loan request has been declined. this is because you do not meet one or more of our eligibility criteria.\n \n - paysense'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'hi sachin, good news! you can now refer your friends to paysense. just open the app here: https://f29yx.app.goo.gl/xqrfvklpu5h9lpzt6 and share your unique link with your friends. for each friend you refer who takes a loan - you and your friend both earn rs. 500. refer more, earn more!'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': "hi muzammil, you're now one step away to get a loan upto 5lacs. check your eligibility now! www.gs.im/n/dtcdalfijsz"},
                      {'loan_app': 'paysense',
                       'category': 'reject',
                       'sms_template': 'based on your updated information, your paysense loan application has been declined. we are sorry we cannot serve you at the moment.\n \n - paysense'},
                      {'loan_app': 'paysense',
                       'category': 'announcement',
                       'sms_template': 'hi heena kamble, we have news for you! lazypay has recently merged with paysense, a reputed personal loan company in india & we would like to invite you to apply for your loan with paysense. it has better policy terms (loan upto 5 lakhs, in 55 cities, quick disbursals) & a support team to guide you through the entire loan journey. check your eligibility now! - www.gs.im/n/ody3tw6h5nx'},
                      {'loan_app': 'paysense',
                       'category': 'apply',
                       'sms_template': "hi madan, please note paysense or its staff will never ask you to make a payment for completing your loan application. don't respond to any calls requesting for payment from your bank account towards processing your loan. in case of any doubt, write to us at support@gopaysense.com"},
                      {'loan_app': 'paysense',
                       'category': 'apply',
                       'sms_template': 'hi m, we have received the documents that you submitted. we will process them soon. - paysense'},
                      {'loan_app': 'paysense',
                       'category': 'apply',
                       'sms_template': "you are just a few steps away from completing your paysense loan application. give us a missed call on 01141185464 and we'll help you complete your loan application."},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'hi boggula, you have been assigned loan amount upto rs. 63000.0 based on the details provided through our partners. download paysense app https://f29yx.app.goo.gl/xqrfvklpu5h9lpzt6 and login to complete the loan application process.'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': "last chance to get an instant personal loan at the lowest interest rate and with minimal documentation. give us a missed call at 01141185464 and we'll help you. hurry up!"},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'congrats! your loan application has been approved for a personal loan of rs.500000 from paysense. click here www.gs.im/n/sryydoq27vt to complete your application.'},
                      {'loan_app': 'paysense',
                       'category': 'reject',
                       'sms_template': 'hi kadali venkata durga prasad, your loan application is rejected because your documents did not meet our requirements. check email for details.'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'congrats! your loan application has been approved as per your credit-assessment on indialends, for a personal loan of rs.310000 from paysense. download the app www.gs.im/n/yqnd6fg5nwq & login to complete your application.'},
                      {'loan_app': 'paysense',
                       'category': 'repaid',
                       'sms_template': 'hi vinod, payment of rs. 1846.0 for your paysense loan was received on 2019-11-05. your loan status will be updated in 2 working hours. click https://f29yx.app.goo.gl/w636shshbsawvmjv2 to know more.'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'mubarak, you are one step away from getting your loan of rs.100000 approved by paysense. complete your application now! click www.gs.im/n/cvwquimpouy'},
                      {'loan_app': 'paysense',
                       'category': 'apply',
                       'sms_template': 'are you salaried or self-employed? complete your loan application on paysense app here www.gs.im/n/nt0r4bppmx & get loan amount of rs.15500 in your bank account!'},
                      {'loan_app': 'paysense',
                       'category': 'apply',
                       'sms_template': 'dear customer! upload your selfie on paysense app to complete your loan application for rs. 21000 on paysense . click here www.gs.im/n/fkvwioudyrx'},
                      {'loan_app': 'paysense',
                       'category': 'apply',
                       'sms_template': 'hi yogesh, please note paysense or its staff will never ask you to share your internet banking password, debit/credit card number, expiry date, cvv or pin for your loan repayment. in case of any doubt, write to us at support@gopaysense.com and we will get back to you.'},
                      {'loan_app': 'paysense',
                       'category': 'apply',
                       'sms_template': 'upload your address proof photo on paysense app to move ahead with your loan application of rs. 37000 on paysense. click here www.gs.im/n/eisggnvr18u'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'dear jwell, paysense knows home is where the heart is and heart is where she is! pamper her with an amazon gift voucher worth rs 2000 on a personal loan of rs 100000. hurry! check \neligibility at https://www.gopaysense.com/login/?utm_source=lp-payu-quantum and claim your voucher now. t&c*applied.'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'हाय jwell, paysense के साथ 5 लाख तक का व्यक्तिगत ऋण प्राप्त करें।""\nन्यूनतम दस्तावेज | तेज़ ऋण प्रक्रिया। और क्या? मुफ्त अमेज़न प्राप्त करें\nयदि आप <https://www.gopaysense.com/login/?utm_source=lp-payu-quantum> पर अभी आवेदन करते हैं, तो ^ ^ राशि का वाउचर।\nकेवल आज के लिए घोषित करें! tnc * लागू किया.'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'hi jwell, check eligibility in 30 seconds for a personal loan of rs 100000 from paysense and get free amazon voucher of rs 1000 hurry! click here to check eligibility https://www.gopaysense.com/login/?utm_source=lp-payu-quantum. t&c*applied'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'hey, get a personal loan in  easy steps, at the lowest interest rate and minimal documentation with paysense right now! www.gs.im/n/kwzpfpxo'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'hi mohan, get personal loan up to  lacs with paysense. minimal documents, faster loan process. what more? get free amazon voucher worth rs  if you apply now at https://www.gopaysense.com/login/?utm_source=lp-payu-quantum\noffer valid only for today! tnc*applied.'},
                      {'loan_app': 'paysense',
                       'category': 'promote',
                       'sms_template': 'dear swapnil sunil, missed your loan offer? on demand from our valued customers, we have extended next loan offer of rs 50000. for next 5 hrs only. hurry now and click at https://goo.gl/fsdux5 .t&c* applied'},
                      {'loan_app': 'gotocash',
                       'category': 'apply',
                       'sms_template': 'dear customer, you are currently applying for a loan with gotocash, your verification code is 865254, valid for 30 minutes, please do not share or disclose.'},
                      {'loan_app': 'gotocash',
                       'category': 'login',
                       'sms_template': 'dear customer, your login verification code for gotocash is 198757, valid for 30 minutes, please do not share or disclose.'},
                      {'loan_app': 'gotocash',
                       'category': 'apply',
                       'sms_template': 'dear customer, your loan application to gotocash has been submitted successfully, please wait for system review!'},
                      {'loan_app': 'gotocash',
                       'category': 'reject',
                       'sms_template': 'dear thati naresh, we are sorry that your loan application to gotocash did not pass the eligibility requirements, you can apply again after 15 days, thank you.'},
                      {'loan_app': 'gotocash',
                       'category': 'approval',
                       'sms_template': 'dear janipalli ashokbabu, congratulations! your loan application to gotocash has been approved, we will transfer the funds to your bank account ending with 7416, please wait patiently.'},
                      {'loan_app': 'gotocash',
                       'category': 'disbursal',
                       'sms_template': 'dear gopinath penta, your loan from gotocash has been disbursed to your account ending with 9306, amounting to 3820 rupees. please note and verify.'},
                      {'loan_app': 'gotocash',
                       'category': 'reminder',
                       'sms_template': 'dear sailaja jyothi, the due date of your loan from gotocash is 3 days from now, total repayment amount is rs 7139.65, please login and repay. if you want to continue to use the loan, you can apply for an extension and extend the loan time.'},
                      {'loan_app': 'gotocash',
                       'category': 'reminder',
                       'sms_template': 'dear sachin rokade , the due date of your loan from gotocash is tomorrow, total repayment amount is rs 5066.5, please login and repay. if you want to continue to use the loan, you can apply for extension and extend the loan time.'},
                      {'loan_app': 'gotocash',
                       'category': 'reminder',
                       'sms_template': 'dear pravin patidar , the due date of your loan from gotocash is today, total repayment amount is rs 5066.5. please login and repay, as failure to repay today will incur additional costs and affect your personal credit. if you want to continue to use the loan, you can apply for extension and extend the loan time.'},
                      {'loan_app': 'gotocash',
                       'category': 'repaid',
                       'sms_template': 'dear chander mohan, your loan from gotocash is paid off now. you could log in the app to apply for another loan.'},
                      {'loan_app': 'gotocash',
                       'category': 'promote',
                       'sms_template': 'need immediate funds? get an instant personal loan with emi as low as rs.2199/month at interest rate starting from 10.99%. click http://p.psbzr.in/pl11'},
                      {'loan_app': 'gotocash',
                       'category': 'promote',
                       'sms_template': 'transfer your outstanding credit card debt to personal loan with interest rate as low as 10.99%. click http://f49.bz/15lx5b to apply'},
                      {'loan_app': 'gotocash',
                       'category': 'promote',
                       'sms_template': "congrats!! get up to rs. 25,000 money in your account within few minutes no paperwork, no collateral's!! apply now http://y2s.in/l.s?e19s6c20"},
                      {'loan_app': 'gotocash',
                       'category': 'extension',
                       'sms_template': 'dear vijay bhimaji gargate, due date extension on your loan from gotocash is successful, new due date is 18-03-2020. please pay on time to enjoy higher credit and other benefits.'},
                      {'loan_app': 'kreditzy',
                       'category': 'login',
                       'sms_template': '843658 is otp for kreditzy login. do not share otp with anyone u8p9zb6z/2q'},
                      {'loan_app': 'kreditzy',
                       'category': 'registration',
                       'sms_template': 'dear tanwer alam, you have successfully signed up on kreditzy! keep your pan card ready to check your loan eligibility. kreditzy.com/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'apply',
                       'sms_template': 'dear ritesh, we have received your loan application! please wait while we verify the details.'},
                      {'loan_app': 'kreditzy',
                       'category': 'apply',
                       'sms_template': '617895 is otp for loan agreement digital signature. do not share code with anyone u8p9zb6z/2q'},
                      {'loan_app': 'kreditzy',
                       'category': 'disbursal',
                       'sms_template': 'congratulations vinay! your loan has been disbursed to your bank account xxxxxxx2188 after deduction of processing fees. repay on time to avoid late payment charges! kreditzy.com/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'reminder',
                       'sms_template': 'dear manmohan, payment of rs. 5150 is due by 2020-04-02. pay on time to avoid penalty of rs. 100. pay via bank transfer/debit card/netbanking/paytm wallet/upi from the options in kreditzy app. select bank transfer option to avoid convenience fees. ignore if paid. https://kreditzy.com/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'reject',
                       'sms_template': 'sorry! we could not approve your profile at this moment. please re-apply in 6 months'},
                      {'loan_app': 'kreditzy',
                       'category': 'apply',
                       'sms_template': 'congratulations manpreet you are eligible for a loan. keep your original address proof and pan cards ready for completing your profile. kreditzy.com/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'repaid',
                       'sms_template': 'dear customer, we have successfully received payment of rs. 4118. please wait upto 24 hours for the amount to reflect in your loan'},
                      {'loan_app': 'kreditzy',
                       'category': 'repaid',
                       'sms_template': 'dear shekh imranbhai, your loan kz200314qmjeu has been closed successfully! kreditzy.com/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'promote',
                       'sms_template': 'congratulations dinesh, your credit has been upgraded! you can now avail loans upto rs. 2200 at kreditzy at lower interest rates. avail an instant loan now!'},
                      {'loan_app': 'kreditzy',
                       'category': 'reminder',
                       'sms_template': 'dear akhil, please ensure that your cibil score is not impacted by paying rs. 4118 today. this will also help you to get more credit on kreditzy platform based on your good payment behavior and avoid late payment charges of rs. 70. ignore if paid. https://kreditzy.com/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'reminder',
                       'sms_template': 'dear kokkirala, payment of rs. 4120 is due tomorrow. pay on time to avoid penalty of rs. 70. pay via bank transfer/debit card/netbanking/paytm wallet/upi from the options in kreditzy app. select bank transfer option to avoid convenience fees. ignore if paid. https://kreditzy.com/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'overdue',
                       'sms_template': 'dear kokkirala, your loan is overdue and charges of rs. 100 have been levied. amount repayable is rs. 5248. pay immediately to avoid further penalty. ignore if paid. https://kreditzy.com/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'overdue',
                       'sms_template': 'dear aanand, please repay your emi amount due on 2020-01-10 as per following details using imps transfer. the details are only for you and transfer will close your loan only. do not share details with anyone.\nbeneficiary name: kartbee tech\nacc. num: 545545004485652\nifsc: yesb0cmsnoc\nremarks: 9712464020\namount: 1674'},
                      {'loan_app': 'kreditzy',
                       'category': 'overdue',
                       'sms_template': 'dear aanand, you can repay your outstanding emi amount of rs. 1674 due on 10-01-2020 using the given repayment link. pay via debit card, netbanking, wallets. convenience fees charges as applicable may apply. click the link to pay. https://krdit.be/kz191226hbzottqrk0'},
                      {'loan_app': 'kreditzy',
                       'category': 'promote',
                       'sms_template': 'dear karmvir, your bank account xxxxxxxxxxx5581 has been verified successfully. submit your loan application for instant loan now! kreditzy.com/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'overdue',
                       'sms_template': 'dear karmvir, your loan contract has a delinquency of rs. 9422 since 2 days. we request you to make the payment of the pending amount at the earliest. please ignore if already paid. https://kreditzy.com/app - team kreditzy'},
                      {'loan_app': 'kreditzy',
                       'category': 'overdue',
                       'sms_template': 'dear karmvir, we would like to inform you that your account has a pending amount of rs. 9522 including late payment charges of rs. 300, which was due on 2020-02-21. we are trying to contact you on your mobile number, but you are not responding our calls. please ignore if already paid. https://kreditzy.com/app - team kreditzy'},
                      {'loan_app': 'kreditzy',
                       'category': 'reminder',
                       'sms_template': 'dear rakesh, rs. 10250 is due today. please pay on time as this will also help you to get more credit on kreditzy platform based on your good payment behavior. ignore if paid. https://kredt.be/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'overdue',
                       'sms_template': 'legal notice alert - dear kyla, this is to inform you that your loan amount of rs. 9772 is overdue and pending payment since 9 days. hence, we request you to make the payment on priority. in case it is not done, you may receive a legal notice from us and we may initiate legal proceedings on your contract as per policy. please ignore if already paid. https://kreditzy.com/app - team kreditzy'},
                      {'loan_app': 'kreditzy',
                       'category': 'approval',
                       'sms_template': 'dear mr./ms. mamidi lalith kumar, pursuant to your loan application form submitted to kreditzy, a loan of the total amount of inr 5000 has been sanctioned by usha financial services private limited to you. please contact us on 08044292200 immediately in case of any disputes'},
                      {'loan_app': 'kreditzy',
                       'category': 'promote',
                       'sms_template': 'congratulations mamidi! you have discovered a lower pricing for your loans. avail an instant loan at kreditzy now! https://kredt.be/app'},
                      {'loan_app': 'kreditzy',
                       'category': 'apply',
                       'sms_template': 'dear merinkumar, we are verifying your profile. please wait upto 24 hours for the process to be completed. we will notify you once your profile is verified.'},
                      {'loan_app': 'kreditzy',
                       'category': 'apply',
                       'sms_template': 'dear merinkumar, your uploaded  aadhaar document(s) were not clear. please try again with a good quality original document. kreditzy.com/app'},
                      {'loan_app': 'zestmoney',
                       'category': 'login',
                       'sms_template': '661152 is the otp for your zestmoney account. do not share the otp with anyone. zestmoney never calls to verify it.'},
                      {'loan_app': 'zestmoney',
                       'category': 'login',
                       'sms_template': '<#> 892280 is the otp for your zestmoney account. do not share it with anyone.\n cb59piwrv5h'},
                      {'loan_app': 'zestmoney',
                       'category': 'registration',
                       'sms_template': 'thanks for choosing zestmoney. keep your aadhaar id, income & bank details ready to experience the fastest way to pay using digital emi. to complete your application follow this - http://zest.care/tfmpc'},
                      {'loan_app': 'zestmoney',
                       'category': 'repaid',
                       'sms_template': 'sunen dutta, you are the best. the payment of ₹1,508.00 for your zestmoney loan#40f864a003d1 was successful. login to your zestmoney account to track your loan https://youraccount.zestmoney.in'},
                      {'loan_app': 'zestmoney',
                       'category': 'registration',
                       'sms_template': 'hi, thank you for signing up with zestmoney! you are just 2 steps away from activating your credit limit and enabling the world of convenience.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'your emi is due in 2 days\n ------\n hi zakiuddin, do not let late payment negatively affect your cibil score. please maintain sufficient balance in your bank account for auto-debit.\n \n your amazonpay loan of inr 1616 is due on 2020-04-17t04:00:00.\n ------\n please ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': 'dear naveen nandakumar,\n  \n since you have not repaid your zestmoney emi, we will be shortly reporting to cibil. please reach us at 09343422556 to settle your dues. login to app.zestmoney.in and pay your zestmoney emi to avoid any penalties. kindly ignore if paid already.'},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': 'dear sujit mahata,\n  \n you have missed your zestmoney emi ,login to app.zestmoney.in and pay your zestmoney emi to avoid any penalties. on time repayments helps in improving your cibil score. for any help call at 093434 22556. kindly ignore if paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': 'dear sameer s,\n  \n we were unable to reach you with reference to your zest emi, please note we have reported your details to cibil. if due payment is made, your record will be updated as "paid" in cibil. kindly reach us on 09343422556 to settle your dues.'},
                      {'loan_app': 'zestmoney',
                       'category': 'registration',
                       'sms_template': "hey there, thanks for signing up with zestmoney, you're just 2 steps away from discovering your credit limit. login to continue https://apps.zestmoney.in"},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 8 days left to make your emi payment\n ------\n hi ankush, your emi of inr 983 on your amazon loan will be auto-debited 2020-01-06t04:00:00.\n \n repay on time to increase your credit score. you can also pay online - http://bit.ly/2xsfrfk\n -----\n please ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 1 day left to make your emi payment\n ------\n\n we will work with your bank to collect your monthly emi. please maintain sufficient funds for auto-debit.\n ------\n please ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'your emi payment is due in 4 days\n ------\n hi partha, please check your bank account & maintain sufficient balance for your emi payment of inr 1069 on flipkart loan.\n \n you can also pay online - http://bit.ly/2xsfrfk \n ------\n please ignore if already paid'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 4 days left to make your emi payment\n ------\n hi kadali, please check your bank account and maintain sufficient balance for your emi payment of inr 733 on amazonpay loan.\n \n you can also pay online - http://bit.ly/2xsfrfk \n ------\n please ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - your emi is due in 7 days\n ------\n hi kadali, your emi of inr 871 on your amazonpay loan will be auto-debited on 2020-03-11t04:00:00. \n \n on time repayment leads to improved cibil score. you can also pay online - http://bit.ly/2xsfrfk\n ------\n please ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': "hi , we noticed that you've cancelled your zestmoney emi application for inr 50000.00 on flipkart. you can login to your account to reapply in case you changed your mind."},
                      {'loan_app': 'zestmoney',
                       'category': 'apply',
                       'sms_template': 'important \n ----- \n hi neetu, your kyc is not complete, please update your kyc details & enjoy the convenience of 0% interest emi* at your favourite stores with zestmoney - http://bit.ly/37qlhrm'},
                      {'loan_app': 'zestmoney',
                       'category': 'repaid',
                       'sms_template': 'gantakumar ., you are the best. the payment of ₹1,977.44 for your zestmoney loan#0336007944ba was successful. login to your zestmoney account to track your loan https://youraccount.zestmoney.in'},
                      {'loan_app': 'zestmoney',
                       'category': 'apply',
                       'sms_template': 'thanks for applying for flipkart voucher of inr 50000. you can always login to your account to check its status - http://bit.ly/2fzg4o9'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 6 days left to make your emi payment\n ------\n hi kadali, please maintain sufficient bank balance as your emi of inr 901 on your amazonpay loan is due on 2020-02-10t04:00:00. \n \n you can also pay online - http://bit.ly/2xsfrfk\n ------\n please ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'apply',
                       'sms_template': 'your emi plan for loan #bbc7efb861f5 has been accepted. get your loan activated by completing your application - http://bit.ly/2fyxhhc'},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': 'dear yash srivastava,\n  \n we were unable to reach you with reference to your zest emi, kindly reach us on 09343422556 to settle your dues, else we will be taking a legal action against you.'},
                      {'loan_app': 'zestmoney',
                       'category': 'apply',
                       'sms_template': "hi , we noticed that you've cancelled your zestmoney emi application for inr 50000.00 on amazon pay gift cards. you can login to your account to reapply in case you changed your mind."},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 3 days left to make your emi payment\n ------\n \n please maintain sufficient balance to avoid nach bounce charges from your bank.\n ------\n please ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'congrats, your amazon pay gift cards worth inr 3258.00 has been approved. you are now ready to shop on amazon and pay later in easy emis.\n \n login to your zestmoney account and click on view vouchers to activate your emi plan - http://bit.ly/2fz5ygz'},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': 'dear rishabh, you can pay rs. 1636.0 for your zestmoney loan #6f4411f8a606 from https://rzp.io/i/fqkm47g'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'thank you for your trust in zestmoney. it will be great if you can share your feedback with the world by adding it to our google review - https://goo.gl/xny6nr'},
                      {'loan_app': 'zestmoney',
                       'category': 'apply',
                       'sms_template': "great, you've accepted the loan agreement. login to your zestmoney account & complete the process to activate your emi plan - http://bit.ly/2fypk7j"},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'hey maneesh, you have an approved credit limit of inr 22000. login to your zestmoney account to start shopping with our partners https://apps.zestmoney.in'},
                      {'loan_app': 'zestmoney',
                       'category': 'approval',
                       'sms_template': 'hey prasen, your account is being reviewed and our decision engine will soon approve & add credit balance into your zestmoney account. login to your account to know the status https://apps.zestmoney.in'},
                      {'loan_app': 'zestmoney',
                       'category': 'approval',
                       'sms_template': 'hi , your amazon pay gift cards of inr 2035.00 is generated. head to amazon to use your gift card now! happy shopping.'},
                      {'loan_app': 'zestmoney',
                       'category': 'approval',
                       'sms_template': 'hi dastagiri, your amazon pay gift cards of inr 1500.00 is now ready! login to your account to avail the gift card. happy shopping!'},
                      {'loan_app': 'zestmoney',
                       'category': 'approval',
                       'sms_template': 'congrats, your amazon pay gift cards worth inr 4479.00 has been sent to your email id rkr_sws010@yahoo.com. happy shopping!'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': '50-80% off on fashion brands at myntra #endofreasonsale. buy now with zestmoney @ 0% interest emi + 10% cashback* (under emi on checkout). http://bit.ly/2menjxx'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'your zestmoney emi application for paytm order of inr 1500 has been cancelled. \n \n help us improve our service by sharing your feedback - http://bit.ly/2rfo7qc'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'hope you had a great experience of using zestmoney emi. on a scale of \n 1-10 how likely are you to recommend us to friends & family? \n http://bit.ly/2yc2vsc'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'get cashback of upto ₹2,000 on availing your first loan with zestmoney! enjoy the convenience of 0% interest emi across amazon, flipkart, and makemytrip. no processing fee or pre-closure charges - http://nmc.sg/diiofu'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 4 days left to make your emi payment\n ------\n hi , your emi payment of inr 568 on xiaomi loan is due on 2020-04-07t04:00:00\n \n on-time emi payments will help you get bigger loans in the future. pay online - http://bit.ly/2xsfrfk\n ------\n please ignore if already paid'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'enjoy the convenience of 0% interest emis on phones, fashion, travel & more with zestmoney. no credit card required | get credit limit - http://bit.ly/2lzb3fg'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': '948247 is your otp to activate your loan, valid for 30 min. please read your complete loan agreement here - http://zest.care/jehws.\n password is your pan number. activation of your loan using the otp shall deem acceptance of the terms of loan agreement.\n do not share the otp with anyone.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 7 days left to make your emi payment\n ------\n \n increase your credit score and credit limit by paying your emi on time. pay online - http://bit.ly/2xsfrfk\n -----\n please ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 2 days left to make your emi payment\n ------\n \n pay online - http://bit.ly/2xsfrfk\n ------\n please ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': "it's amazingly simple to purchase things on your wishlist on emi. start shopping by completing your income verification on zestmoney. http://bit.ly/2myhtvs"},
                      {'loan_app': 'zestmoney',
                       'category': 'registration',
                       'sms_template': 'hi santosh, zestmoney employees will not ask for your otp or any other sensitive information. please do not share your details under any circumstance.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 10 days left to make your emi payment\n ------\n \n make your payment now and stay worry free. pay online - http://bit.ly/2xsfrfk\n -----\n please ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'extension',
                       'sms_template': 'dear rakesh dholpuriya,\nthank you for opting out of pausing your emi payments for dcfdf818a8f5. your emi payment schedule will proceed as originally scheduled. \n\nalso, you stand a chance to win cashback upto inr 4000 if you pre-close your loan. to view your eligible cashback amount and pre-close your loans, log into my account: http://youraccount.zestmoney.in'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'hey ratneshwar, your next emi will be debited on tuesday, 07/04/2020. \n\npaying your emis on time is the safest way to avoid extra costs. \n\nlog in to https://bit.ly/3dotwdx if you think you are in financial distress or will have difficulty paying.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': '#zestmoney bumper offer -  pay next 3 emis on time & stand a chance win inr 5000 worth amazon gift card.\n\npay now - https://bit.ly/2vqqcy0'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'अनुस्मारक - आपके ईएमआई का भुगतान करने के लिए 6 दिन बचे हैं\n------\nनमस्ते , कृपया अपने बैंक खाते में पर्याप्त पर्याप्त शेष राशि रखें क्योंकि आपके amazon लोन पर inr 1125 की ईएमआई 6 दिनों में देय है। समय पर पुनर्भुगतान से सिबिल स्कोर में सुधार होता है, जो अन्य वित्तीय उत्पादों जैसे कि होम या कार लोन का लाभ उठाने के लिए महत्वपूर्ण है। \n\nआप ऑनलाइन भुगतान अभी तुरंत कर सकते हैं - http://bit.ly/2rpjogz\n\nकृपया ध्यान दें कि यदि आपका ईएमआई भुगतान हमें देय तारीख से 3 दिन पहले प्राप्त हो जाता है, तो हम ऑटो डेबिट रोक देंगे।\n------\nअगर पहले ही भुगतान कर दिया है, तो कृपया इसपर ध्यान न दें।'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'अनुस्मारक - आपके ईएमआई का भुगतान करने के लिए सिर्फ 1 दिन बचा है\n ------\n नमस्ते , आपके amazon लोन के लिए inr 1131 की आपकी ईएमआई कल देय है। \n \n हम आपके मासिक ईएमआई का संग्रह करने के लिए आपके बैंक के साथ काम करेंगे। कृपया ऑटो-डेबिट के लिए पर्याप्त राशि बनाए रखें।\n \n क्या आप जानते हैं कि समय पर पुनर्भुगतान आपके क्रेडिट स्कोर को बेहतर बनाने में मदद करता है? एक स्वस्थ सीबिल स्कोर कार या होम लोन जैसे अन्य वित्तीय उत्पादों का लाभ उठाने के लिए महत्वपूर्ण है। \n ------\n अगर पहले ही भुगतान कर दिया है, तो कृपया इसपर ध्यान न दें।'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 1 day left to make your emi payment\n------\n\nwe will work with your bank to collect your monthly emi. please maintain sufficient funds for auto-debit.\n------\nplease ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reject',
                       'sms_template': "very sorry pardeep kumar, we're unable to approve the emi plan for your amazon order at this time. this can be for a number of reasons which we've emailed you."},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "emi pending\n-------\nhi , as per our records, you're yet to pay to your emi of ₹1247 which was due on 2020-02-10t00:00:00 for your order #cb6c780ac013. \n-------\ndon't negatively impact your cibil score, you need good score for future loans. pay your emi on time. \n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "awaiting your emi payment\n-------\nhi , we're yet to receive your emi payment of ₹1620 was due on 2020-02-10t00:00:00 for your order #dfdc94989dc0. \n-------\nany further delay in emi payments will result in freezing your credit limit. so don't miss your emi payments. \n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': 'awaiting your emi payment\n-------\nhi , our records show that we have not received your emi payment of ₹1745 which was due on 2020-02-17t00:00:00 for your order #4e5p7013t02p. \n-------\nnon-payment of emi freezes your credit limit, this will hinder you from getting future loans. \n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "awaiting your emi payment\n-------\nhi nagaraja s, our records show that we have not received your emi payment of ₹2251 which was due on 2020-02-03t00:00:00 for your order #ze78962b2344. \n-------\na cash-based personal loan is most helpful, don't delay emi payments or you will lose your chance of getting one.\n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'hey vinit, your next emi will be debited on tuesday, 07/04/2020. \n\npaying your emis on time is the safest way to avoid extra costs. \n\nlog in to https://bit.ly/3dotwdx if you think you are in financial distress or will have difficulty paying.'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': "flipkart's #supercoolingdays is live | upto 60% off on acs, refrigerators, air coolers & more. hurry, get your voucher today @ 0% interest on 3 & 6 months emi. also, get upto ₹2000 cashback transferred to your bank account on your first loan - http://nmc.sg/dlqp0i"},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'hey prasad, your next emi will be debited on tuesday, 07/04/2020. \n\npaying your emis on time is the safest way to avoid extra costs. \n\nlog in to https://bit.ly/3dotwdx if you think you are in financial distress or will have difficulty paying.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 4 days left to make your emi payment\n------\nhi mohd, your emi payment of inr 2616 on personalloan loan is due on 2019-12-19t04:00:00\n\non-time emi payments will help you get bigger loans in the future. pay online - http://bit.ly/2xsfrfk\n------\nplease ignore if already paid'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 10 days left to make your emi payment   \n ------ \nhi pankaj, your emi payment of ₹1449 on flipkart loan is due in 10 days.\n------\npay online - http://bit.ly/2rpjogz\n------\ntimely emi payment increases your credit score for higher eligibility on future loans.\n------\nplease ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - your emi is due in 1 day\n------\nhi d, please make sure to pay your emi of inr 1211 is due tomorrow for your amazon loan.\n\nmaking late payments will negatively affect your credit score. pay online - http://bit.ly/2xsfrfk\n------\nplease ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - your emi is due in 7 days\n------\nhi uma, your emi of inr 5489 on your personalloan loan will be auto-debited on 2020-02-27t04:00:00. \n\non time repayment leads to improved cibil score. you can also pay online - http://bit.ly/2xsfrfk\n------\nplease ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - your emi is due today\n------\nhi prashant, your emi of inr 850 for your flipkart loan will be auto-debited from your bank account today. \n\nnon maintenance of balance for emi deduction will lead to levy of penalties by your bank.\n-----\nplease ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder\n------\nhi shuaib ahmed, on time emi payments not only help increase your credit score but also the eligibility of higher loan amounts in the future. \nyour emi of ₹540 on order #80935858c288 is due in 10 days. don’t miss it and pay on time. \n------\npay online \nhttp://nmc.sg/k6cqnz\nif we receive the emi payment 3 days before the due date, we will stop the auto debit.\n------\nplease ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'apply',
                       'sms_template': 'thanks for applying for flipkart voucher for amount ₹15000. you can always login https://app.zestmoney.in to check its status.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reject',
                       'sms_template': 'hi ankit, the loan amount you have applied currently exceeds your credit limit. please apply for a loan within your available credit limit of inr 10000 to enjoy the convenience of zestmoney emis.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reject',
                       'sms_template': "we're unable to process your loan as your application did not meet the eligibility criteria. kindly use a different mode of payment for your flipkart transactions."},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'hi pankaj, your credit limit validity has expired. please login to your zestmoney account and verify your details to activate your credit limit - http://bit.ly/2pdvkeb'},
                      {'loan_app': 'zestmoney',
                       'category': 'announcement',
                       'sms_template': 'your feedback is important to us! on a scale of 1-10 how satisfied are you with the shopping experience on flipkart using zestmoney emi? http://bit.ly/2vw3tmp'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'hi neeraj, your flipkart voucher of inr 8500.00 is generated. head to flipkart to use your voucher now! happy shopping.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reject',
                       'sms_template': 'hi vishal verma, your previous emi application of rs.3300 for your has been cancelled in order to process your current application.'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'zestmoney can help you own what you want from your favourite merchant in easy emis & also get a cashback of upto ₹2000 transferred to your bank account on your first loan. get your voucher today @ 0% interest on 3 & 6 months plan - http://nmc.sg/d5x3b5'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'अनुस्मारक - आपके ईएमआई का भुगतान करने के लिए 10 दिन शेष हैं\n------\nनमस्ते sumit, समय पर ईएमआई का भुगतान करना न केवल आपके क्रेडिट स्कोर को बढ़ाने में मदद करता है, बल्कि भविष्य में उच्च ऋण राशि की पात्रता हासिल करने में भी आपकी मदद करेगा।\n\nआपके flipkart ऋण पर आपका inr 8532 की ईएमआई की देय तिथि 10 दिनों में बीत जाने वाली है। समय पर भुगतान करना न चूकें।\n\nआप ऑनलाइन भुगतान अभी तुरंत कर सकते हैं - http://bit.ly/2rpjogz\n\nयदि हमें आपका ईएमआई भुगतान देय तिथि से 3 दिन पहले मिल जाता है, तो हम ऑटो डेबिट रोक देंगे।\n------\nअगर पहले ही भुगतान कर दिया है, तो कृपया इसपर ध्यान न दें।'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'अनुस्मारक - आपके ईएमआई का भुगतान करने के लिए सिर्फ 1 दिन बचा है\n ------\n नमस्ते kunal, कृपया अपने inr 708 के ईएमआई का समय पर भुगतान करना सुनिश्चित करें, जो आपके flipkart लोन के लिए कल देय है।\n \n अभी ऑनलाइन अपना भुगतान करें - http://bit.ly/2rpjogz\n \n क्या आप जानते हैं कि देर से भुगतान करने से आपके क्रेडिट स्कोर पर नकारात्मक प्रभाव पड़ेगा? \n ------\n अगर पहले ही भुगतान कर दिया है, तो कृपया इसपर ध्यान न दें।'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'dear karthikeyan,\n\nmaking your emi payments has never been more rewarding! get scratchcards and win incredible cashbacks upto inr 1000 within 24 hours of every timely payment you make towards your emis!\n\ncalculate the additional interest amount that you can avoid paying by opting out of the moratorium here: http://zemo.us/tfolr\n\nfor single step opt-out, give us a missed call at 080-47109893'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': "hi sudhakar, pay your emi today & get cashback up to inr 200. emi payments have never been so rewarding with cashback + chance to increase credit scores & get higher credit eligibility.\n\nyour emi of inr 2360 is due on '3/4/2020 12:00:00 am'.\n\nwhy wait? pay now & win cashback!  - https://rzp.io/i/fy0ww6j\n----\nplease ignore if paid or have already set up auto debit."},
                      {'loan_app': 'zestmoney',
                       'category': 'repaid',
                       'sms_template': 'hi anil reddy gottumukkala, thank you for paying your zestmoney emi for loan id #0649d8409997. as promised, we have refunded the interest amount of inr 151 to your bank account.'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'hi suresh, you are opted in for pausing your monthly emis. \n\nmaking your emi payments has never been more rewarding! get scratchcards and win incredible cashbacks upto inr 500 within 24 hours of every timely payment you make towards your emis! \n\nopt-out of pausing your emis and make the payment now by logging into my account - https://bit.ly/2e5ambh'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'hi heena, your feedback is important to us! on a scale of 0-10, how satisfied are you with the zestmoney emi payment experience with regards to your amazonpay loan powered by dmi finance? http://bit.ly/2rf6efj'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'hi sudhakar, your next emi will be debited on saturday, 04/04/2020. rbi has allowed moratorium to benefit customers in severe financial distress. if you wish to opt-in for moratorium (emi pause), please visit myaccounts - https://bit.ly/2e5ambh'},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "awaiting your emi payment\n-------\nhi ramagiri saikumar, your emi of ₹10097 for order #acc6c94d101c was due on 2020-02-17t00:00:00. however, we're yet to receive the emi payment. \n-------\nyou will be eligible for bigger and better zestmoney loans in the future if you pay your emis on time.\n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'approval',
                       'sms_template': 'congrats, your flipkart order worth inr 22000.00 has been approved. you are now ready to shop on flipkart and pay later in easy emis. \n\nlogin to your zestmoney account and click on view vouchers to activate your emi plan - http://bit.ly/2fz5ygz'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'congrats, your flipkart voucher code worth inr 1500.00 has been sent to your email id shrutimishr863@gmail.com. happy shopping!'},
                      {'loan_app': 'zestmoney',
                       'category': 'approval',
                       'sms_template': 'congrats! your emi application of inr 17284 on flipkart has been successfully processed. flipkart will notify you with the product delivery details. for your emi details, login to your zestmoney account at http://bit.ly/2e5ambh'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'dear zestmoney customer.\n\nwe were glad to be of help today. share your experience below\nhttps://zestmoneysupport.typeform.com/to/huymvr'},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "emi pending\n-------\nhi dhruba ghosh, we're yet to receive your emi payment of ₹1127 which was due on 2020-02-07t00:00:00 for your order #88f22c26908d.\n-------\nno one wants to be denied a personal loan, don't be that customer. pay your emi's on time. \n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'emi reminder \n------\nhi pankaj kothari, your emi of \n₹2026 is due on \n05/10/2019 for your \nflipkart loan.\n------\npay online - http://bit.ly/2unundl\n------\ndid you know timely emi payments improve your credit score!\n------\nplease ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': "emi reminder\n-------\nhi sanjay, at times payments can be missed. hence, this is a reminder that we're yet to receive your emi payment"},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "emi reminder\n-------\nhi sandeep singh, at times payments can be missed. hence, this is a reminder that we're yet to receive your emi payment of ₹499 which was due on 2020-02-17t00:00:00 for your order #a02b29996902. \n-------\nkeep in mind that timely payment of emis will ensure that your cibil score is high. you can get a home loan or a car loan easily in the future, if you have a good cibil score. \n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "emi reminder\n-------\nhi nishant, at times payments can be missed. hence, this is a reminder that we're yet to receive your emi payment of ₹697 which was due on 2019-07-15t00:00:00 for your order #9aeb935917a5. \n-------\nwe want to help you better your cibil score but late payments will only have a negative impact. pay your emis on time.\n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "emi reminder\n-------\nhi brinda chivukula, at times payments can be missed. hence, this is a reminder that we're yet to receive your emi payment of ₹141 which was due on 2020-02-20t00:00:00 for your order #b0b7b84da755. \n-------\ndon't make the mistake of missing an emi payment, this will affect your cibil score and it will be difficult for you to get a loan in the future. \n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "emi reminder\n-------\nhi maruti uppar, we're yet to receive your emi payment of ₹2318 which was due on 2020-02-18t00:00:00 for your order #aa60fcb9d59c.\n-------\neveryone needs financial help sometimes, pay your emi's on time to avoid your credit limit from freezing. \n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "emi reminder\n-------\nhi venkata tanuku, we're yet to receive your emi payment of ₹1718 which was due on 2020-02-05t00:00:00 for your order #3ba45b339b08.\n-------\nfailure to make timely repayments may lead to freezing your credit limit! \n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'registration',
                       'sms_template': '914886 is your moneed register otp. for security, please do not share it with anyone. it is available in 10 minutes'},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': 'emi reminder\n-------\nhi rathika, your emi of ₹360 remains unpaid. it was due on 2020-02-05t00:00:00 for your order #cd905f7a611c. \n-------\nthe best way to maintain a high cibil score is to make your emi payments on time! a delay in payment will be reported to cibil on a regular basis and will affect your cibil score.\n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'अनुस्मारक - आपके ईएमआई का भुगतान करने के लिए 8 दिन बचे हैं\n ------\n नमस्ते umesh, क्या आप जानते हैं कि समय पर ईएमआई का भुगतान करने से आपके क्रेडिट स्कोर में सुधार होता है? उच्च क्रेडिट स्कोर होने पर भविष्य में आप आसानी से लोन प्राप्त कर सकते हैं। \n \n आपके amazonpay ऋण पर आपका inr 629 की ईएमआई की देय तिथि 8 दिनों में बीत जाने वाली है।\n \n आप ऑनलाइन भुगतान अभी तुरंत कर सकते हैं - http://bit.ly/2rpjogz\n \n कृपया ध्यान दें कि यदि आपका ईएमआई भुगतान हमें देय तारीख से 3 दिन पहले प्राप्त हो जाता है, तो हम ऑटो डेबिट रोक देंगे। \n ------\n अगर पहले ही भुगतान कर दिया है, तो कृपया इसपर ध्यान न दें।'},
                      {'loan_app': 'zestmoney',
                       'category': 'overdue',
                       'sms_template': "awaiting your emi payment\n-------\nhi nagaraja s, we have not received your emi of ₹2001 which was due on 2020-02-03t00:00:00 for your [merchant_name] order #ze78962b2344. \n-------\ndon't make the mistake of missing an emi payment, this will affect your cibil score and it will be difficult for you to get a loan in the future. \n-------\npay online via upi, wallet, internet banking or debit card.\nhttps://youraccount.zestmoney.in\n-------\nplease ignore if already paid."},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'dear nagaraja,\n\nmaking your emi payments has never been more rewarding! get scratchcards and win incredible cashbacks with every timely payment you make towards your emis! opting out of moratorium can also help you save the additional interest you will have to pay otherwise.\n\nopt-out of pausing your emis and make the payment now by logging into my account: http://youraccount.zestmoney.in'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'dear nagaraja,\nexciting news! get up to inr 4000 cashback upon the pre-closure of your outstanding loans with zestmoney! to know your eligible cashback amount and pre-close your loans, log into my account: http://youraccount.zestmoney.in'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'dear nagaraja,\nmaking your emi payments has never been more rewarding! get scratchcards and win incredible cashbacks upto inr 1000 within 24 hours of every timely payment you make towards your emis! also, continuing your emi payments will help you save more money in the long run.\ncalculate the additional interest amount that you can avoid paying by opting of the moratorium here: http://zest.care/tfolr'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'dear bittu yadav, you are currently opted-in for pausing your loan payments to zestmoney.\n\npaying off your loan has never been more fun or rewarding! your original payment amount is 1070 but you only have to pay 970 by making payment towards your personal loan today.\n\nyour eligible discount is already applied so payment takes mere seconds. also, opting out of moratorium can help you save the additional interest you will have to pay otherwise. \n\nmake your loan payment in a flash through the following link - https://rzp.io/i/lyb6znz'},
                      {'loan_app': 'zestmoney',
                       'category': 'reminder',
                       'sms_template': 'reminder - 2 days left to make your emi payment\n------\nhi kunal, please make sure to pay your emi of inr 219 on or before 2020-02-01t04:00:00 for your makemytrip loan.\n\npay online - http://bit.ly/2xsfrfk\n------\nplease ignore if already paid.'},
                      {'loan_app': 'zestmoney',
                       'category': 'promote',
                       'sms_template': 'hi nagaraja, making your emi payments has never been more rewarding! get scratchcards and win incredible cashbacks with every timely payment you make towards your emis! opting out of moratorium can also help you save the additional interest you will have to pay otherwise.\n\nopt-out of pausing your emis and make the payment now by logging into my account - https://bit.ly/2e5ambh'},
                      {'loan_app': 'moneed',
                       'category': 'apply',
                       'sms_template': '617318 is the otp code for the loan contract you confirmed in moneed. for security,please do not share it with anyone.'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': '668552 is your momo register otp. for security, please do not share it with anyone. it is available in 10 minutes'},
                      {'loan_app': 'moneed',
                       'category': 'reject',
                       'sms_template': 'sorry, your loan application was rejected due to a comprehensive assessment, please keep a good credit history and apply again after 60 days.'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': '922825 is the otp for password on moneed. for security, please do not share it to others. it is available in 10 minutes.'},
                      {'loan_app': 'moneed',
                       'category': 'promote',
                       'sms_template': 'india vs bangladesh, 2nd test, chances to win upto 25lakhs* welcome bonus rs. 500* register now : https://bit.ly/2kqwwzu'},
                      {'loan_app': 'moneed',
                       'category': 'promote',
                       'sms_template': 'india vs bangladesh, 3rd t20, chances to win upto 25lakhs* welcome bonus rs. 500* register now : https://bit.ly/2kqwwzu'},
                      {'loan_app': 'moneed',
                       'category': 'approval',
                       'sms_template': 'sorry your bank account info is incorrect, in case of failure disbursement, pls modify your bank info through latest app version on xxx ( 01/06/2019) ¤äbankcardclosingmodifydateñ. pls ensure all account are correct and you are real bank account holder'},
                      {'loan_app': 'moneed',
                       'category': 'promote',
                       'sms_template': 'cricket 2nd odi india vs west indies receive rs 500/- bonus make your team & play now: https://bit.ly/2kqwwzu'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': '847535 is the otp for password on momo. for security, please do not share it to others. it is available in 10 minutes.'},
                      {'loan_app': 'moneed',
                       'category': 'reject',
                       'sms_template': 'sorry your application were rejected due to internal policy. you can re-apply after 7 working days.'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': 'your verification code is 701507.'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': 'your loan has been overdue for 16 days! daily 2% of the loan amount will be charged. pay now to avoid further legal action and effect to your credit score.'},
                      {'loan_app': 'moneed',
                       'category': 'approval',
                       'sms_template': 'congrats，your loan application has been approved. and your loan disbursement was success. loan tenure : 7 days. due date is 2019-11-13. you may need repay rs.10000.00 on time, otherwise it will affect your cibil score.'},
                      {'loan_app': 'moneed',
                       'category': 'repaid',
                       'sms_template': 'payment received, your moneed loan was closed successfully. thanks for your support!'},
                      {'loan_app': 'moneed',
                       'category': 'reminder',
                       'sms_template': 'your loan will due in 3 days, please repay in time through moneed app. good repayment record can maintain your credit score. any issues call customer service at +918929366777.'},
                      {'loan_app': 'moneed',
                       'category': 'reminder',
                       'sms_template': 'your loan will due tomorrow, please repay in time through moneed app. good repayment record can maintain your credit score. any issues call customer service at +918929366777.'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': 'your loan has been overdue for 7 days! daily 2% of the loan amount will be charged. we advice you can apply for extension repay to avoid much penalty, effect on your credit score or further legal action.'},
                      {'loan_app': 'moneed',
                       'category': 'approval',
                       'sms_template': '427252 is the otp code for the loan contract you confirmed in momo. for security,please do not share it with anyone.'},
                      {'loan_app': 'moneed',
                       'category': 'login',
                       'sms_template': '172392 is your moneed login otp. for security,please do not share it with anyone.'},
                      {'loan_app': 'moneed',
                       'category': 'reminder',
                       'sms_template': 'your loan is due today. please repay in time through moneed app. or you can apply for extension repay to avoid overdue charges and further penalty. any issues call customer service at +918929366777.'},
                      {'loan_app': 'moneed',
                       'category': 'disbursal',
                       'sms_template': 'congrats! your withdrawal is successful. loan amount rs 4000.0, repayment date: 2020/03/11. you need to pay the amount rs 4000.0 on time. otherwise, it will affect your credit score.'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': 'your loan is overdue 1 days! daily 2% penalty fees of the loan amount will be charged. current total repayment amount is rs.3105. we advice you can repay as soon as possible to avoid affecting your credit score. click www.app.momomomo.in to repay immediately.'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': "your loan is overdue 3 days. we tried to contact you, but you didn't repay or didn't pick up the call, so we will contact your families or friend to remind you. pls repay before 4 pm today."},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': 'your loan is overdue for 7 days! repay it before 4 pm today, or legal action will be taken, and your credit score will be influenced.'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': "your loan is overdue for 15 days! we'll turn this case to our attorneys, you'll be forced to repay the loan as well as court penalty fees rs30,000"},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': 'your loan has been overdue for 17 days! daily 2% of the loan amount will be charged. due to the inconvenience of covid-19, our company decided to relieve penalty fees (overdue charges) to help you getting through. the reduction deadline until april 15. we advice you can repay as soon as possible.'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': "your loan has been overdue for 25 days! daily 2% of the loan amount will be charged.we tried to contact you, but you're uncooperative or do not pick up the call, so we will contact your families or friend to remind you. pay now to avoid further legal action and effect to your credit score."},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': "your loan is overdue for 28 days! we'll exposure your overdue info on newspaper and social media. everyone will see it after searching your name."},
                      {'loan_app': 'moneed',
                       'category': 'reminder',
                       'sms_template': 'your loan with amount rs.3000, the withdrawal date 2020-03-08 will due in 1 day. current total repayment amount is rs.3045. you can click  www.app.momomomo.in for repayment. a good repayment record will boost your credit score.'},
                      {'loan_app': 'moneed',
                       'category': 'reminder',
                       'sms_template': 'your loan with amount rs.3000, the withdrawal date 2020-03-08 is due today. current total repayment amount is rs.3045. you can click  www.app.momomomo.in for repayment. a good repayment record will boost your credit score.'},
                      {'loan_app': 'moneed',
                       'category': 'repaid',
                       'sms_template': 'thanks we’ve received your payment rs 2300. current repayable amount your loan with amount rs 3000, withdrawal date 2020-03-08 is rs985'},
                      {'loan_app': 'moneed',
                       'category': 'announcement',
                       'sms_template': 'dear customers, club factory will never ask for your bank details, including your card number, cvv & otp etc to offer you gifts, lucky prizes or refund settlement. kindly ensure you never share any such information over inbound calls, sms, e-mail, claiming to be from club factory.'},
                      {'loan_app': 'moneed',
                       'category': 'login',
                       'sms_template': '[4fun app] 6166 (login code),effective within 5 minutes'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': '[minijoy] your confirmation code is: 7254'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': '[minijoy] आपका पुष्टि कोड 7717 है।'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': '[shareit] 5368 is your confirmation code, ten-minute validity.'},
                      {'loan_app': 'moneed',
                       'category': 'promote',
                       'sms_template': '#india vs sl final match\nmake your team & win upto 25 lakhs\n play now\nhttps://bit.ly/2kqwwzu'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': '<#> 983358 is your verification code, enter it on yoyo\nnskc7vngfa3'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': '<#> otp for your mobile verification is 4352. thanks finnable.com jnrg9r7su//'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': '<#> weshare 2913 is your verification code. please login in five minutes. 3+hmdc34ivk'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': 'äminijoyñ your confirmation code is: 7392'},
                      {'loan_app': 'moneed',
                       'category': 'promote',
                       'sms_template': 'big bash league t20 matches\n\nget welcome bonus  rs 500/- \n\nmake your team & play now: ç\nhttps://bit.ly/2kqwwzu'},
                      {'loan_app': 'moneed',
                       'category': 'approval',
                       'sms_template': 'congrats! your loan application has been approved, this credit is valid for 60 days. pls withdrawal it through www.app.momomomo.in'},
                      {'loan_app': 'moneed',
                       'category': 'promote',
                       'sms_template': 'congratulations and welcome to fonesafe classic! your membership no. is im4536636. please note our helpline no. 18602583030 (mon-sun, 11am-9pm). thank you, cpp india'},
                      {'loan_app': 'moneed',
                       'category': 'announcement',
                       'sms_template': 'file overdue returns and regularise gst or income tax or tds compliance before 31st march 2020 through indiafilings.com at an affordable price. call 04440247777'},
                      {'loan_app': 'moneed',
                       'category': 'announcement',
                       'sms_template': 'file overdue gstr-1 before 10th jan to avoid penalty. gst and tds returns are due in january 2020. for help with efiling, contact indiafilings.com 8822667788'},
                      {'loan_app': 'moneed',
                       'category': 'registration',
                       'sms_template': '993556 is your otp for gamezop login. happy playing!'},
                      {'loan_app': 'moneed',
                       'category': 'approval',
                       'sms_template': 'loan disbursement was failed, if you still need, pls download latest app version and modify your bank account info. if the info still incorrect, your application will be closed. you will receive sms , email or app notification about next apply time.any issues call customer service at +.'},
                      {'loan_app': 'moneed',
                       'category': 'repaid',
                       'sms_template': 'thanks we have received your payment rs3165. your loan with amount rs 3000, withdrawal date 2020-03-02 has been closed.'},
                      {'loan_app': 'moneed',
                       'category': 'extension',
                       'sms_template': 'today is your loan extension date, and you can pay the extension processing fee through the app to get an extension; if you have paid or settled the loan, please ignore it.'},
                      {'loan_app': 'moneed',
                       'category': 'extension',
                       'sms_template': 'your extension repay application is approval!charge the extension fee rs. 2619.00 and the extension repay will reflect soon. updated repay amount rs. 10000.00, repay date:02/04/2020'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': 'your loan is overdue 46 days! daily 2% penalty fees of the loan amount will be charged. current total repayment amount is rs.5805. we advice you can repay as soon as possible to avoid further legal action & affect on your credit score. click  www.app.momomomo.in to repay immediately.'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': 'your loan is overdue 3 days! repayment amount is rs.4300. if we can not receive your payment at 4 pm today, we will contact your friends and family to remind you. click  www.app.momomomo.in to repay.'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': 'your loan is overdue 7 days! repayment amount is rs.4620.if we can not receive your payment at 4 pm today, it will affect on your credit score and further legal action will be taken. click  www.app.momomomo.in to repay.'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': "your loan is overdue 36 days! daily 2% penalty fees of the loan amount will be charged. current total repayment amount is rs.8675. we tried to contact you, but you're uncooperative or do not pick up the call, so we will contact your families or friend to remind you. pls repay before 4 pm today to avoid further legal action & affect on your credit score. click  www.app.momomomo.in to repay immediately."},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': "your loan is overdue for 3 days! if we can't receive the repayment after 4pm today, we'll start contacting with your relatives."},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': 'your loan with principal amount rs 5000.00 is overdue 60 days. we sincerely remind you to keep reliable behaviors and repay the loan positively. please feel free to contact service team service@moneed.net for further assistance.'},
                      {'loan_app': 'moneed',
                       'category': 'overdue',
                       'sms_template': 'your loan with total principal amount rs 4000.0 is overdue 16 days. we sincerely remind you to keep reliable behaviors and repay the loan positively. please feel free to contact service team service@moneed.net for further assistance.'}]

    @classmethod
    def __adjust(cls):
        templates_df = pd.DataFrame(cls.templates_list)
        templates_df['sms_template'] = templates_df['sms_template'].apply(
            lambda x: re.sub("dear(.*?),|[0-9]", "", x))
        category = {}
        templates = {}
        for ind, sms_sr in templates_df.iterrows():
            app = sms_sr.get('loan_app')
            sms_tplt = sms_sr.get('sms_template')
            sms_ctg = sms_sr.get('category')

            if app not in templates.keys():
                app_df = templates_df[templates_df['loan_app'] == app]
                templates[app] = list(app_df['sms_template'])

            if app not in category.keys():
                category[app] = {}

            category[app][sms_tplt] = sms_ctg
        return templates, category

    @classmethod
    def __adjust_phone_regex(cls):
        return {phone.get('loan_app'): phone.get('phone_regex') for phone in cls.app_phone_list}

    @property
    def category_config(self) -> dict:
        if self._category_config is None:
            self._template_config, self._category_config = self.__adjust()
        return self._category_config

    @property
    def template_config(self) -> dict:
        if self._template_config is None:
            self._template_config, self._category_config = self.__adjust()
        return self._template_config

    @property
    def phone_regex(self) -> dict:
        if self._phone_regex is None:
            self._phone_regex = self.__adjust_phone_regex()
        return self._phone_regex


class SMSLoanHistoryAnalyse:
    SMSLoanHistoryAnalysisConfig = SMSLoanHistoryAnalysisConfig()
#     default_columns_ls = [
#         'smsSendTime', 'smsSenderName', 'smsSenderPhone', 'smsType',
#         'smsReceiverName', 'smsReceiverPhone', 'smsContent', 'category', 'loanAppName', 'days',
#         'amount', '_id'
#     ]
    default_columns_ls = [
        'smsSendTime', 'smsSenderName', 'smsSenderPhone', 'smsType',
           'smsContent', 'category', 'loanAppName', 'days',
        'amount'
    ]
    # default_output_columns_ls = ['smsSendDate', *default_columns_ls]
    default_output_columns_ls = [*default_columns_ls]
    overdue_regex = re.compile(
        r"(?<!avoid any) legal|be blocked|is overdue|(?<!in) \d+ day|(?<!with) \d+ day|(?<!last) \d+ day|(?<!only) \d+ day")
    days_regex = re.compile(r".? (\d+) day")
    amount_regex = re.compile(
        '(?:rs|inr|₹|rp)\.?\s?(?P<money1>[1-9][0-9,]+(?:\.[0-9][0-9])?)|.? (?P<money2>[1-9][0-9,]+(?:\.[0-9][0-9])?) rp')

    loan_name = {
        'credicxo', 'mystro', 'speedcash', 'loanfront', 'moneytap', 'revfin', 'dhani', 'finserv',
        'yelo', 'udhaar', 'cashe', 'earlysalary', 'instabiz', 'cashmama', 'vcard', 'robocash', 'branch app',
        'faircent', 'kyepot', 'rupeek', 'loansimple', 'rapidrupee', 'fleximoney', 'branchmanager',
        'cashcred', 'moneyenjoy', 'okcash', 'indialends', 'flashcash', 'rupeelend', 'loanfit',
        'crazyrupee', 'nira', 'badabro', 'lendingadda', 'credime', 'paisaonline', 'finnable', 'rupeemax',
        'loanbaba', 'starloan', 'instasalary', 'loanit', '10minuteloan', 'cashbus', 'kissht', 'uucash',
        'yaarii', 'flashpaisa', 'lendkaro', 'rupeebox', 'sahukar', 'apnapaisa', 'persoloan',
        'loanwix', 'rubique', 'speedycash', 'dancerupee', 'cashpalm', 'snapcash', '5nance', 'checkpoint',
        'cashkumar', 'smartloan', 'stashfin', 'offermeloan', 'cashbean', 'instaloan', 'kreditbee',
        'credit2u', 'flexsalary', 'upwards', 'kreditzy', 'cashin', 'loanle', 'erupee', 'paisabazaar',
        'moneyloji', 'dealsofloan', 'goeasy', 'gorupee', 'rupeebus', 'cashbucks', 'iobnanban', 'loantap',
        'creditt', 'finwego', 'mpokket', 'rupeeclub', 'wificash', 'shubhloans', 'gotocash',
        'instamoney', 'phoneparloan', 'manappuram', 'smartcoin', 'icredit', 'loanadda', 'quikkloan',
        'myloanmitra', 'pocketloan', 'muthootblue', 'flexiloans', 'mudrakwik', 'cashday', 'loanwalle',
        'lendingkart', 'lendenclub', 'lazypay', 'moneyview', 'loanflix', 'moneymore', 'cashclub',
        'ieasyloan', 'etmoney', 'moneed', 'hicash', 'doublecash', 'cashbook', 'rupeecash', 'cashpapa',
        'moneywow', 'micredit', 'abcash', 'ycash', 'shubh', 'sweetrupee', '360loan', 'anytimeloan',
        'atomecredit', 'cashbeen', 'cashbowl', 'creditbee', 'easyloan', 'zestmoney',
        'freeloan', 'friendloan', 'getrupee', 'goldcash', 'goldenlightning', 'homecredit', 'flipkart',
        'indiabullsdhani', 'irupee', 'kreditone', 'loanbro', 'loanbus', 'loandost', 'loanrecord',
        'loantime', 'madrupee', 'moneybox', 'moneyinminutes', 'moneyviewloans', 'morerupee', 'mpocket',
        'nanocard', 'nanocred', 'nanocredit', 'ocash', 'paisapay', 'paymeindia', 'paysense',
        'personalloan', 'rupeefast', 'rupeeloan', 'rupeeplus', 'rupeeredee', 'rupeestar', 'salarydost',
        'silvercredit', 'truebalance', 'ucash', 'cashbn', 'cashwin', 'payme ', 'kisst',
        'creditcash', 'cash+', 'kisht', 'fastrupee', 'creditzy', 'easylone', 'casebeen', 'mpoket',
        'cashnw', 'moneyday', 'casebean', 'cashln', 'lonetime', 'happymonies', 'cashup', 'oyeloans',
        'cashtm', 'cashloan', 'loanmart', 'cashbin', 'supercash', 'superrupee', 'lonefront', 'firstapp',
        'mobikwik', 'billioncash', 'cashmore', 'cashpie', 'quickcredit', 'silverkredit', 'landkaro',
        'freelone', 'kishht', 'yesrupee', 'casenow', 'kist', 'rupbus', 'idfc', 'easyloan', 'rbl',
        'paisapay', 'easycashloan', 'airtel', 'bajaj', 'flexi'
    }

    def __init__(self, df: pd.DataFrame, phone_cfg: dict = None):
        self.df = df
        if phone_cfg is None:
            phone_cfg = self.SMSLoanHistoryAnalysisConfig.phone_regex
        self.phone_cfg = phone_cfg
        self.analyzed_df = None

    @staticmethod
    def find_in_set(keys: [str], str_set: set, return_hit_key: bool = False):
        for key in keys:
            if key in str_set:
                return key if return_hit_key else True
        return None if return_hit_key else False

    @classmethod
    def _get_days(cls, x):
        """提权短信里面关于天数的字段"""
        days_finds = cls.days_regex.search(x)
        return int(days_finds.group(1)) if days_finds else np.NaN

    @classmethod
    def _get_amount(cls, x):
        """提取短信里面的金额字段"""
        amount_finds = cls.amount_regex.search(x)
        if not amount_finds:
            return np.nan

        amount_str = (amount_finds.group('money1') or amount_finds.group('money2'))
        return float(amount_str.replace(',', ''))

    def sms_general_category(self, df: pd.DataFrame, selected_cols: list = None) -> pd.DataFrame:
        """通用短信分类器"""
        if selected_cols is None:
            selected_cols = self.default_columns_ls

        if df.empty:
            return pd.DataFrame([], columns=selected_cols)

        df_copy = df.copy(deep=True)
        df_copy['smsContent'] = df_copy['smsContent'].str.lower()

        # 数据处理
        df_copy = df_copy.apply(self._general_filter, axis=1)
        df_copy['is_loan_overdue'] = df_copy['is_loan_overdue'].astype(bool)
        df_copy['is_loan_app'] = df_copy['loanAppName'].astype(bool)

        # category 归类
        tags = pd.Series(data='other', index=df_copy.index)
        tags[(df_copy.is_loan_app == 1) & (df_copy.is_otp == 1)] = 'registration'  # 注册
        tags[(df_copy.is_loan_app == 1) & (df_copy.is_promote == 1)] = 'promote'  # 促销短信
        tags[(df_copy.is_loan_app == 1) & (df_copy.is_otp == 0) & (df_copy.is_loan_approve == 1)] = 'approval'  # 通过
        tags[(df_copy.is_loan_app == 1) & (df_copy.is_otp == 0) & (df_copy.is_loan_reject == 1)] = 'reject'  # 拒绝
        tags[(df_copy.is_loan_app == 1) & (df_copy.is_otp == 0) & (df_copy.is_loan_paid == 1)] = 'repaid'  # 还款成功
        # 如果击中还款提醒，就是还款提醒，否则是逾期
        tags[(df_copy.is_loan_app == 1) & (df_copy.is_loan_overdue_or_reminder == 1)] = 'overdue'
        tags[(df_copy.is_loan_app == 1) & (df_copy.is_otp == 0) & (df_copy.is_loan_reminder == 1) & (
                df_copy.is_promote == 0) & (df_copy.is_loan_approve == 0)] = 'reminder'  # 还款提醒
        tags[(df_copy.is_loan_app == 1) & (df_copy.is_loan_overdue == 1)] = 'overdue'  # 逾期
        df_copy['category'] = tags
        return df_copy[selected_cols]

    @classmethod
    def _general_filter(cls, df_copy):
        """字段筛选器"""
        x = df_copy['smsContent']
        x_set = set(re.split('[ :,.!?()]', x))

        df_copy['is_otp'] = cls.find_in_set(['<#>', 'otp', 'verification'], x_set)

        df_copy['is_promote'] = cls.find_in_set(['apply for', 'apply now', 'welcome bonus', 'upto', 'coupons'], x) or (
                'get' in x_set and 'http' in x_set)

        df_copy['is_loan_overdue_or_reminder'] = cls.find_in_set(
            ['not paid', 'levied', 'past due', 'overdue', 'over due', 'over-due', 'not recieved'], x)

        df_copy['is_loan_overdue'] = cls.overdue_regex.search(x)

        app_names = cls.loan_name.intersection(x_set)
        df_copy['loanAppName'] = list(app_names)[0] if app_names else ''

        df_copy['is_loan_reject'] = cls.find_in_set(["hasn't been approved", "not be processed", "can't be processed"],
                                                    x)

        df_copy['is_loan_paid'] = 'paid off' in x

        df_copy['is_loan_approve'] = cls.find_in_set(['accepted', 'put into your account', 'has been approved'], x)

        df_copy['is_loan_reminder'] = \
            cls.find_in_set(['reminder', 'overdue fee', 'repayment date', 'penalty', 'do not overdue', 'warning',
                             'credit score', 'is due', 'today', 'tomorrow', 'due soon', 'ignore'], x)

        # df_copy['is_link'] = cls.find_in_set(['http', '.in/', '.in ', '.com/', '.com '], x)
        # df_copy['is_loan'] = cls.find_in_set(['loan', 'emi'], x_set)

        df_copy['days'] = cls._get_days(x)

        df_copy['amount'] = cls._get_amount(x)

        return df_copy

    def sms_template_category(self, df: pd.DataFrame, selected_cols: list = None) -> tuple:
        """
        按照模板进行分类
        :param df:
        :param selected_cols: 选择的列名
        :return:
        """
        # 预处理
        if selected_cols is None:
            selected_cols = self.default_columns_ls

        if df.empty or 'smsSenderPhone' not in df.columns:
            return pd.DataFrame([], columns=selected_cols), df

        df_copy = df.copy(deep=True)
        df_copy['smsContent'] = df_copy['smsContent'].str.lower()
        df_copy['loanAppName'] = self.match_app_name(df_copy)
        df_copy_has_loan_app = df_copy[~df_copy['loanAppName'].isnull()]

        # 匹配模版并分类
        # 1. 按照app获取短信最匹配的模板
        # 2. 按照模板提取短信对应的分类
        df_loan_app_all_res = [pd.DataFrame([], columns=selected_cols)]
        for app in set(df_copy_has_loan_app['loanAppName']):
            df_each_app = df_copy_has_loan_app[df_copy_has_loan_app['loanAppName'] == app]

            df_each_app['sms_template'] = self.sms_template_match(
                df_each_app, self.SMSLoanHistoryAnalysisConfig.template_config[app])

            df_each_app['category'] = df_each_app['sms_template'].replace(
                self.SMSLoanHistoryAnalysisConfig.category_config[app])

            df_loan_app_all_res.append(df_each_app)

        df_loan_app_all_res = pd.concat(df_loan_app_all_res, axis=0, sort=True)

        df_loan_app_all_res['days'] = df_loan_app_all_res['smsContent'].apply(self._get_days)
        df_loan_app_all_res['amount'] = df_loan_app_all_res['smsContent'].apply(self._get_amount)

        df_loan_app_res = df_loan_app_all_res[df_loan_app_all_res['category'] != NOT_HIT_TEMPLATE_KEY]

        not_match_filter = (df_copy['loanAppName'].isnull()) | (df_loan_app_all_res['category'] == NOT_HIT_TEMPLATE_KEY)
        df_category_null_res = df_copy.loc[not_match_filter,]

        return df_loan_app_res, df_category_null_res

    def match_app_name(self, df: pd.DataFrame) -> pd.Series:
        """
        按照smsSenderPhone字段匹配出对应的app名称
        :param df:
        :return:
        """
        df_copy = df.copy(deep=True)

        df_copy['loanAppName'] = None
        for app_key in self.phone_cfg.keys():
            phone_regex = self.phone_cfg.get(app_key)
            is_app_phone = df_copy['smsSenderPhone'].str.match(phone_regex, na=False)
            df_copy.loc[is_app_phone, 'loanAppName'] = app_key

        return df_copy['loanAppName']

    def sms_template_match(self, df: pd.DataFrame, template_cfg: list) -> pd.Series:
        df_copy = df.copy(deep=True)
        df_copy['sms_template'] = df_copy['smsContent'].apply(lambda x: self.__fuzz_extract_best(x, template_cfg))
        return df_copy['sms_template']

    @staticmethod
    def __fuzz_extract_best(a_string: str, compare_ls: list, scorer=fuzz.ratio, score_cutoff: int = 80):
        a_string = re.sub("([0-9])|(dear.*?,)", "", a_string)
        hit_best = process.extractOne(
            a_string,
            compare_ls,
            scorer=scorer,
            score_cutoff=score_cutoff
        )

        if not hit_best:
            return NOT_HIT_TEMPLATE_KEY

        return hit_best[0]

    def sms_category(self, need_templates: bool = True) -> pd.DataFrame:
        """

        :param need_templates:
        :return:
        """
        df_copy = self.df.copy(deep=True)
        if df_copy.empty:
            return pd.DataFrame([], columns=self.default_output_columns_ls)

        df_template_category = pd.DataFrame()

        if need_templates:
            df_template_category, df_other = self.sms_template_category(df_copy)

        else:
            df_other = df_copy

        df_other_category = self.sms_general_category(df_other)

        df_result = pd.concat([df_template_category, df_other_category], axis=0, sort=True)

        # 补充字段
        df_result['loanAppName'].replace(' ', '', inplace=True)
        # df_result['smsSendDate'] = df_result['smsSendTime'].dt.date
        df_result = df_result[self.default_output_columns_ls]

        return df_result.sort_values(by=['smsSendTime']).reset_index(drop=True)

    def get_df(self):
        if self.df.empty:
            return pd.DataFrame(columns=self.default_output_columns_ls)
        if not self.analyzed_df:
            self.analyzed_df = self.sms_category(need_templates=False)

        return self.analyzed_df.copy(deep=True)



In [20]:

def get_sms():
    # tdf = pd.read_csv('/Users/dengwei/Downloads/deviceUserSMS.csv')
    tdf = pd.read_csv('record.csv')
    tdf.rename(columns = {
            'smsInfo.smsSendTime':'smsSendTime',
            'smsInfo.smsContent':'smsContent',
            'smsInfo.smsSenderName':'smsSenderName',
            'smsInfo.smsSenderPhone':'smsSenderPhone',
            'smsInfo.smsType':'smsType' ,
            'smsInfo.smsReceiverName':'smsReceiverName',
            'smsInfo.smsReceiverPhone':'smsReceiverPhone'
        }, inplace = True)
    return tdf

sms = get_sms()

In [21]:
print(sms[["smsContent","smsSenderPhone","smsType", "loanApp", "loanStatus","days", "amount"]].head())

                                          smsContent smsSenderPhone  smsType  \
0  CASHe has requested payment of INR 7000. You c...      AX-RZRPAY  receive   
1  CASHe has requested payment of INR 7000. You c...      AX-RZRPAY  receive   
2  StashFin has requested payment of INR 1725. Yo...      MD-RZRPAY  receive   
3  CASHe has requested payment of INR 7000. You c...      AX-RZRPAY  receive   
4  CASHe has requested payment of INR 7000. You c...      AX-RZRPAY  receive   

    loanApp loanStatus  days  amount  
0     cashe      other   NaN  7000.0  
1     cashe      other   NaN  7000.0  
2  stashfin      other   NaN  1725.0  
3     cashe      other   NaN  7000.0  
4     cashe      other   NaN  7000.0  


In [22]:
print(sms[["smsContent"]].head().to_string())

                                                                                            smsContent
0     CASHe has requested payment of INR 7000. You can pay through this link: https://rzp.io/i/mASeYaa
1     CASHe has requested payment of INR 7000. You can pay through this link: https://rzp.io/i/A3s6Hp9
2  StashFin has requested payment of INR 1725. You can pay through this link: https://rzp.io/i/jtjykWA
3     CASHe has requested payment of INR 7000. You can pay through this link: https://rzp.io/i/ri2RWCj
4     CASHe has requested payment of INR 7000. You can pay through this link: https://rzp.io/i/XjAVaPV


In [ ]:
# sms=sms[:5000]
sms_with_category = SMSLoanHistoryAnalyse(sms).get_df()
# output = 'result.csv'
# sms_with_category.to_csv(output)
# ------
# result = pd.read_csv(output)
print(sms_with_category.category.value_counts()/(len(sms_with_category)) )

In [ ]:
sms.head()

In [ ]:
import copy 
df = copy.deepcopy(sms)
print(len(df))
df.head()

In [ ]:

cates=df.loanStatus.unique()
ss = {}
for i, item in enumerate(cates):
    ss[item] = i
ss
apps = {}
loanApp=df.loanApp.unique()    
for i, item in enumerate(loanApp):
    apps[item] = i
apps

In [ ]:
ss_back = {}
for i, item in enumerate(cates):
    ss_back[i] = item
# ss_back

In [ ]:
feature_col= ['loanApp']
# feature_col= []
text_col = 'smsContent'
y_cate = 'loanStatus'
all_cols =  [text_col]  + feature_col
all_cols_excl_content = all_cols[1:]
all_cols

In [ ]:
df['loanApp'].replace(apps, inplace=True)
df['loanApp']

In [ ]:
from  rf_text_cate_util import *
# import importlib
# rf_text_cate_util = importlib.reload(rf_text_cate_util)

df[text_col]=df[text_col].apply(lambda x: rep_digit(x)) 
df[text_col]=df[text_col].apply(lambda x: tokenize(x)) 
X=df[all_cols]
y=df[y_cate]


In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

X_train_vect,X_test_vect = tf_train(df ,X_train,X_test, text_col, all_cols_excl_content)
X_train_vect.head()

In [ ]:
rf_model,y_pred = text_report(X_train_vect,X_test_vect,y_train, y_test)

In [ ]:
# y_test
y_col=y_cate
df_check = pd.DataFrame(y_test)
df_check['pred'] = y_pred
df_check[all_cols] = X_test
df_check[(df_check[y_col] != df_check['pred'] ) ].to_csv('./pred_diff.csv')


In [ ]:

# output_diff(X_test,y_test,y_pred, y_cate)